In [ ]:
import sys

from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
import pandas as pd
import numpy as np
import gymnasium as gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import os

class StockDataset(Dataset):
    def __init__(self, factors, returns, sequence_length, N=1):
        """
        Parameters:
        - factors: pd.DataFrame of normalized input features (index: date, columns: features)
        - returns: pd.DataFrame with 1 column, normalized returns (index aligned with factors)
        - sequence_length: number of lookback days for attention model
        - N: number of future days to compute mean return for target (default = 1)
        """
        assert factors.shape[0] == returns.shape[0], "Factors and returns must be aligned"

        self.factors = factors.values
        self.returns = returns.values
        self.sequence_length = sequence_length
        self.N = N

    def __len__(self):
        return len(self.factors) - self.sequence_length  # since you're predicting 1 day ahead


    def __getitem__(self, idx):
        X = self.factors[idx : idx + self.sequence_length]  # Shape: (seq_len, num_features)
        y = self.returns[idx + self.sequence_length]  # Scalar target: next-day return
        return torch.tensor(X, dtype=torch.float32), torch.tensor([y], dtype=torch.float32)  # Shape: (1,)


# Example usage
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "2005-01-01"
end_date = "2024-12-31"
output_csv = os.path.join(input_dir, "stock_prices.csv")


factors = pd.read_csv(f"{input_dir}/aligned_factors.csv", index_col=0, parse_dates=True)
returns_raw = pd.read_csv(f"{input_dir}/daily_returns_DIA.csv", index_col=0, parse_dates=True)


# Remove NaN values and align dates
factors = factors.dropna()
returns = returns_raw.dropna()

# Align dates between factors and returns
dates = factors.index.intersection(returns.index)
factors_raw = factors.loc[dates]
returns_raw = returns.loc[dates]
factors_raw = factors_raw.iloc[:, :-1]
# print(f"factors_raw = {factors_raw}")

Nday_diff = 1  # Define the future prediction horizon for shift days
# limit to 2 stock ONLY
returns_raw = returns_raw.iloc[:, :1]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()
# factors_raw = pd.concat([factors_raw, returns_raw], axis=1)
# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

# Shift returns to generate target variable (N-day future return)
returns_raw_Y = returns_raw.shift(-Nday_diff).dropna()


# Align all datasets with available dates in returns_raw_Y
dates_final = factors_raw.index.intersection(returns_raw_Y.index)
factors_raw = factors_raw.loc[dates_final]
returns_raw = returns_raw.loc[dates_final]
returns_raw_Y = returns_raw_Y.loc[dates_final]

N = 1
sequence_length = 10
batch_size = 64
num_epochs = 500
learning_rate = 0.001
rolling_window_train = 252  # Reduce for faster adaptation
rolling_window_test = 10    # Reduce test window
prediction_window = 10      # Align prediction period with test window
train_model2 = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []
all_test_losses = []
all_binary_test_losses = []
results = []
classification_results = []
shap_metrics_results = []
test_results = []
prediction_results = []
combined_test_result = []
portfolio_records = []
start = 0  # Initialize rolling split index

# Rolling split: Train (M days), Test (N days), Predict (W days)
# total_samples = len(factors_raw)
total_samples = 274
# total_samples = 382
print(f'total_samples = {total_samples}')
print(f'total sum = {rolling_window_train + rolling_window_test + prediction_window}')

while start + rolling_window_train + rolling_window_test + prediction_window <= total_samples:

    train_start = start
    train_end = start + rolling_window_train

    test_start = train_end
    test_end = test_start + rolling_window_test

    predict_start = test_end  # Predictions start right after testing
    predict_end = predict_start + prediction_window

    # Extract the actual date indices
    train_dates = (factors_raw.iloc[train_start].name, factors_raw.iloc[train_end - 1].name)
    test_dates = (factors_raw.iloc[test_start].name, factors_raw.iloc[test_end - 1].name)
    predict_dates = (factors_raw.iloc[predict_start].name, factors_raw.iloc[predict_end - 1].name)

    print(f"\n🔹 Processing Date Window: Train {train_dates}, Test {test_dates}, Predict {predict_dates}")
    print(f'returns_raw.columns = {returns_raw.columns}')
    for stock in returns_raw.columns:
        print(f"Training for stock: {stock}")
        stock_returns = returns_raw_Y[[stock]]  # Select the stock column
        # stock_returns = returns_raw[[stock]].shift(-1).dropna()
        # stock_returns_mean = returns_raw_Y[[stock]].rolling(window=N, min_periods=N).mean().shift(-N+1) # calc average over next N period
        # stock_returns = stock_returns_mean.dropna()

        stock_returns_unshift = returns_raw[[stock]]
            
        factors_raw_X = pd.concat([factors_raw, stock_returns_unshift], axis=1)
        # Rolling split: Train (M days), Test (N days), Predict (W days)
        # total_samples = len(factors_raw_X)
        # Initialize the rolling split index

        # Print the selected date ranges
        print(f"Training dates: {train_dates}")
        print(f"Testing dates: {test_dates}")
        print(f"Prediction dates: {predict_dates}")

        # # Split data into training, testing, and prediction sets
        train_factors = factors_raw_X.iloc[train_start:train_end]
        train_returns = stock_returns.iloc[train_start:train_end]
        test_factors = factors_raw_X.iloc[test_start:test_end]
        test_returns = stock_returns.iloc[test_start:test_end]
        predict_factors = factors_raw_X.iloc[predict_start:predict_end]
        predict_returns = stock_returns.iloc[predict_start:predict_end]
        #
        # # Normalize train and test data
        train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
        train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()
        # print(f'train_factors_mean ={train_factors_mean}')
        # Avoid division by zero
        train_factors_std.replace(0, 1e-8, inplace=True)
        train_returns_std.replace(0, 1e-8, inplace=True)

        train_factors_norm = (train_factors - train_factors_mean) / train_factors_std
        train_returns_norm = (train_returns - train_returns_mean) / train_returns_std
        test_factors_norm = (test_factors - train_factors_mean) / train_factors_std
        test_returns_norm = (test_returns - train_returns_mean) / train_returns_std
        predict_factors_norm = (predict_factors - train_factors_mean) / train_factors_std
        predict_returns_norm = (predict_returns - train_returns_mean) / train_returns_std
        # print(f'train_factors_norm dim: {train_factors_norm.shape}')
        # print(f'train_returns_norm dim: {train_returns_norm.shape}')
        dataset = StockDataset(train_factors_norm, train_returns_norm, sequence_length, N=1)
        

        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        # for batch_x, batch_y in dataloader:
        #     print(f"Batch X Shape: {batch_x.shape}")  # Expected: (batch_size, sequence_length, num_features)
        #     print(f"Batch Y Shape: {batch_y.shape}")  # Expected: (batch_size, N, num_features)
        #     break  # Only print for the first batch
        # # Model initialization
        d_model = train_factors.shape[1]
        # print(f'd_model: {d_model}')

        num_heads = min(4,
                        d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
        while d_model % num_heads != 0:
            num_heads -= 1
        # print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
        model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=sequence_length, N=1).to(
            device)
        #
        # # Optimizer and loss function
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
        criterion = nn.SmoothL1Loss(beta=1.0)  # Huber Loss for stability
        #
        # # Training the model
        # print(f"Training window: {train_start} to {train_end}")
        # print(f"Expected FC Layer Input Shape: {sequence_length * d_model}")
        # print(f"Flattened Tensor Shape: {batch_size, sequence_length * d_model}")

        train_model_with_logging(model, dataloader, optimizer, criterion, scheduler, num_epochs, device)

        print(f'Finished training model 1')
        # print(f'train_factors_norm dim: {train_factors_norm.shape}')
        # Convert to PyTorch tensors
        # print(f'test_factors_tensor: {test_factors_tensor.shape}')
        # Run inference on test data

        augmented_train_features, shap_metrics_train = generate_augmented_features(
            train_factors_norm, train_factors_mean, train_factors_std, model, rolling_window_test, device,
            return_metrics=True
        )
        shap_metrics_train["Stock"] = stock
        shap_metrics_train["Start_Date"] = train_dates[0]
        shap_metrics_train["End_Date"] = train_dates[1]
        shap_metrics_train["Phase"] = 'Train'
        shap_metrics_results.append(shap_metrics_train.iloc[0].tolist())

        augmented_test_features, shap_metrics_test = generate_augmented_features_testphase(
            test_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
            model, prediction_window, device, return_metrics=True
        )
        shap_metrics_test["Stock"] = stock
        shap_metrics_test["Start_Date"] = test_dates[0]
        shap_metrics_test["End_Date"] = test_dates[1]
        shap_metrics_test["Phase"] = 'Test'
        shap_metrics_results.append(shap_metrics_test.iloc[0].tolist())

        augmented_predict_features, shap_metrics_predict = generate_augmented_features_testphase(
            predict_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
            model, prediction_window, device, return_metrics=True
        )
        shap_metrics_predict["Stock"] = stock
        shap_metrics_predict["Start_Date"] = predict_dates[0]
        shap_metrics_predict["End_Date"] = predict_dates[1]
        shap_metrics_predict["Phase"] = 'Predict'
        shap_metrics_results.append(shap_metrics_predict.iloc[0].tolist())
        print(f'---Append Shape value metrics for {stock} and last date is {predict_dates[1]}---')




    # start += prediction_window
    start += 1

shap_metrics_results_df = pd.DataFrame(shap_metrics_results, columns=[
            "mean_abs_shap_Mkt-RF", "mean_abs_shap_SMB", "mean_abs_shap_HML", "mean_abs_shap_RMW", "mean_abs_shap_CMA",
            "mean_abs_shap_Stock",
            "mean_shap_Mkt-RF", "mean_shap_SMB", "mean_shap_HML", "mean_shap_RMW", "mean_shap_CMA", "mean_shap_Stock",
            "shap_std_Mkt-RF", "shap_std_SMB", "shap_std_HML", "shap_std_RMW", "shap_std_CMA", "shap_std_Stock",
            "mean_over_std_Mkt-RF", "mean_over_std_SMB", "mean_over_std_HML", "mean_over_std_RMW", "mean_over_std_CMA",
            "mean_over_std_Stock",
            "mean_abs_over_std_Mkt-RF", "mean_abs_over_std_SMB", "mean_abs_over_std_HML", "mean_abs_over_std_RMW",
            "mean_abs_over_std_CMA", "mean_abs_over_std_Stock",
            "feature_importance_ranking_Mkt-RF", "feature_importance_ranking_SMB", "feature_importance_ranking_HML",
            "feature_importance_ranking_RMW", "feature_importance_ranking_CMA", "feature_importance_ranking_Stock",
            "Stock", "Start_Date", "End_Date", "Phase"
        ])
shap_metrics_results_df.to_csv("shap_value_metrics_export_DIA.csv", index=False)
print("shap_value_metrics_export saved successfully!")

In [ ]:
import pandas as pd
import numpy as np
import ta

# Load DIA price data (replace with your actual path)
etf = pd.read_csv("stock_prices_DIA.csv", parse_dates=["Date"])
etf.set_index("Date", inplace=True)

dia = etf[[stock]].copy()  # ✅ returns DataFrame
dia.rename(columns={stock: "Close"}, inplace=True)  # Rename to 'Close'

# Compute indicators
dia["Return_1d"] = dia["Close"].pct_change()
dia["Return_5d"] = dia["Close"].pct_change(5)
dia["Rolling_Return_21d"] = dia["Close"].pct_change(21)
dia["Volatility_5d"] = dia["Return_1d"].rolling(window=5).std()
dia["Volatility_21d"] = dia["Return_1d"].rolling(window=21).std()
dia["Drawdown"] = dia["Close"] / dia["Close"].cummax() - 1

dia["SMA_20"] = ta.trend.sma_indicator(dia["Close"], window=20)
dia["SMA_50"] = ta.trend.sma_indicator(dia["Close"], window=50)
dia["EMA_12"] = ta.trend.ema_indicator(dia["Close"], window=12)
dia["EMA_26"] = ta.trend.ema_indicator(dia["Close"], window=26)
dia["MACD"] = ta.trend.macd_diff(dia["Close"])
dia["RSI_14"] = ta.momentum.rsi(dia["Close"], window=14)

bb = ta.volatility.BollingerBands(close=dia["Close"], window=20)
dia["BB_high"] = bb.bollinger_hband()
dia["BB_low"] = bb.bollinger_lband()
atr = ta.volatility.AverageTrueRange(high=dia["Close"] * 1.01,
                                     low=dia["Close"] * 0.99,
                                     close=dia["Close"])
dia["ATR_14"] = atr.average_true_range()

# Reset index
dia.reset_index(inplace=True)

# Load SHAP metrics
shap_df = pd.read_csv("shap_value_metrics_export_DIA.csv")
shap_df["End_Date"] = pd.to_datetime(shap_df["End_Date"])

# Merge
merged = pd.merge(shap_df, dia, how="left", left_on="End_Date", right_on="Date")
merged.drop(columns=["Date"], inplace=True)
merged.to_csv("shap_with_dia_indicators.csv", index=False)
print("✅ Saved: shap_with_dia_indicators.csv")

In [ ]:
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)


In [ ]:
import pandas as pd
import numpy as np
import ta

# Load DIA price data (replace with your actual path)
etf = pd.read_csv("stock_prices_DIA.csv", parse_dates=["Date"])
etf.set_index("Date", inplace=True)

dia = etf[[stock]].copy()  # ✅ returns DataFrame
dia.rename(columns={stock: "Close"}, inplace=True)  # Rename to 'Close'

In [ ]:
# Step 1: Convert test factors to NumPy and reshape
test_factors_np = train_factors_norm.values.astype('float32').reshape(243, rolling_window_test, 6)

d_model = test_factors_np.shape[2]  # Number of features

# Step 2: Prepare background data for SHAP (using training factors)
# background_data = train_factors_norm.values.astype('float32').reshape(-1, rolling_window_test, d_model)
# background_data_tensor = torch.tensor(background_data, dtype=torch.float32).to(device)


# Step 3: Ensure background_data is a PyTorch tensor
# if not isinstance(background_data, torch.Tensor):
#     background_data_tensor = torch.tensor(background_data, dtype=torch.float32).to(device)

train_factors_np = train_factors_norm.values.astype('float32')
background_windows = [
    test_factors_np[i:i + rolling_window_test]
    for i in range(len(test_factors_np) - rolling_window_test + 1)
]

In [ ]:
background_windows = [
    test_factors_np[i:i + rolling_window_test]
    for i in range(len(test_factors_np) - rolling_window_test + 1)
]

background_data = np.stack(background_windows[:10])  # limit to 10 for SHAP speed
background_data_tensor = torch.tensor(background_data, dtype=torch.float32).to(device)
# background_windows = []
# train_factors_np = train_factors_norm.values.astype('float32')
# for i in range(0, len(train_factors_np) - rolling_window_test + 1):
#     window = train_factors_np[i: i + rolling_window_test]
#     background_windows.append(window)
#
# background_data_tensor = torch.tensor(np.stack(background_windows[:10]), dtype=torch.float32).to(device)

# Step 4: Initialize SHAP GradientExplainer
explainer = shap.GradientExplainer(model, background_data_tensor)

# Step 5: Compute SHAP values for test features
shap_values = explainer.shap_values(torch.tensor(test_factors_np, dtype=torch.float32).to(device))

# Step 6: Process SHAP values (squeeze unnecessary dimensions)
shap_values_squeezed = np.squeeze(shap_values[0])  # Shape: (rolling_window_test, d_model)

# Step 7: Normalize SHAP values using training statistics
shap_values_normalized = (shap_values_squeezed - train_factors_mean.values) / train_factors_std.values

# Step 8: Concatenate normalized SHAP values with test features
augmented_features = np.concatenate([test_factors_np[0], shap_values_normalized],
                                    axis=1)  # Shape: (rolling_window_test, d_model * 2)

In [ ]:
test_factors_norm

In [ ]:
background_data_tensor = torch.tensor(np.stack(background_windows[:25]), dtype=torch.float32).to(device)
background_data_tensor.shape

In [ ]:
# Step 4: Initialize SHAP GradientExplainer
explainer = shap.GradientExplainer(model, background_data_tensor)

# Step 5: Compute SHAP values for test features
shap_values = explainer.shap_values(
    torch.tensor(test_factors_np, dtype=torch.float32).to(device)
)

# Step 6: Process SHAP values (squeeze unnecessary dimensions)


In [ ]:
shap_values_squeezed = np.squeeze(shap_values[0])  # Shape: (rolling_window_test, d_model)
shap_values_squeezed

In [ ]:
# Step 7: Normalize SHAP values using training statistics
shap_values_normalized = (shap_values_squeezed - train_factors_mean.values) / train_factors_std.values

# Step 8: Concatenate normalized SHAP values with test features
augmented_features = np.concatenate([test_factors_np[0], shap_values_normalized],
                                    axis=1)  # Shape: (rolling_window_test, d_model * 2)


# Compute SHAP metrics
shap_metrics = {
    "mean_abs_shap": np.abs(shap_values_squeezed).mean(axis=0),
    "mean_shap": shap_values_squeezed.mean(axis=0),
    "shap_std": np.var(shap_values_squeezed, axis=0),
    "mean_over_std": shap_values_squeezed.mean(axis=0) / np.std(shap_values_squeezed, axis=0),
    "mean_abs_over_std": np.abs(shap_values_squeezed).mean(axis=0) / np.std(shap_values_squeezed, axis=0),
    "feature_importance_ranking": np.argsort(-np.abs(shap_values_squeezed).mean(axis=0)),
    "shap_value_distribution": shap_values_squeezed,
    "top_factors_per_sample": np.argsort(-np.abs(shap_values_squeezed), axis=1)[:, :3],
    "shap_interaction_values": explainer.shap_interaction_values(background_data_tensor) if hasattr(explainer,  'shap_interaction_values') else None,
    "cumulative_shap_impact": np.sum(np.abs(shap_values_squeezed), axis=1),
    # "shap_feature_correlation": np.corrcoef(test_factors_np.T, shap_values_squeezed.T)[:d_model, d_model:],
    "shap_trend": np.mean(shap_values_squeezed, axis=1),
}

metrics_flat = {}
for key in ["mean_abs_shap", "mean_shap", "shap_std", "mean_over_std","mean_abs_over_std", "feature_importance_ranking"]:
    for i, value in enumerate(shap_metrics[key]):
        metrics_flat[f"{key}_{i + 1}"] = value
shap_metrics_df = pd.DataFrame([metrics_flat])
shap_metrics_df

In [ ]:
shap_values_squeezed

In [ ]:
# Step 1: Convert test factors to NumPy and reshape
test_factors_np = test_factors_norm.values.astype('float32').reshape(1, rolling_window_test, -1)
d_model = test_factors_np.shape[2]  # Number of features

# Step 2: Prepare background data for SHAP (using training factors)
# background_data = train_factors_norm.values.astype('float32').reshape(-1, rolling_window_test, d_model)
# background_data_tensor = torch.tensor(background_data, dtype=torch.float32).to(device)


# Step 3: Ensure background_data is a PyTorch tensor
# if not isinstance(background_data, torch.Tensor):
#     background_data_tensor = torch.tensor(background_data, dtype=torch.float32).to(device)

background_windows = []
train_factors_np = train_factors_norm.values.astype('float32')
for i in range(0, len(train_factors_np) - rolling_window_test + 1):
    window = train_factors_np[i: i + rolling_window_test]
    background_windows.append(window)

background_data_tensor = torch.tensor(np.stack(background_windows[:10]), dtype=torch.float32).to(device)

# Step 4: Initialize SHAP GradientExplainer
explainer = shap.GradientExplainer(model, background_data_tensor)

# Step 5: Compute SHAP values for test features
shap_values = explainer.shap_values(torch.tensor(test_factors_np, dtype=torch.float32).to(device))

# Step 6: Process SHAP values (squeeze unnecessary dimensions)
shap_values_squeezed = np.squeeze(shap_values[0], axis=(0, -1))  # Shape: (rolling_window_test, d_model)

# Step 7: Normalize SHAP values using training statistics
shap_values_normalized = (shap_values_squeezed - train_factors_mean.values) / train_factors_std.values

# Step 8: Concatenate normalized SHAP values with test features
augmented_features = np.concatenate([test_factors_np[0], shap_values_normalized],
                                    axis=1)  # Shape: (rolling_window_test, d_model * 2)

# Compute SHAP metrics
shap_metrics = {
    "mean_abs_shap": np.abs(shap_values_squeezed).mean(axis=0),
    "mean_shap": shap_values_squeezed.mean(axis=0),
    "shap_std": np.var(shap_values_squeezed, axis=0),
    "mean_over_std": shap_values_squeezed.mean(axis=0) / np.std(shap_values_squeezed, axis=0),
    "mean_abs_over_std": np.abs(shap_values_squeezed).mean(axis=0) / np.std(shap_values_squeezed, axis=0),
    "feature_importance_ranking": np.argsort(-np.abs(shap_values_squeezed).mean(axis=0)),
    "shap_value_distribution": shap_values_squeezed,
    "top_factors_per_sample": np.argsort(-np.abs(shap_values_squeezed), axis=1)[:, :3],
    "shap_interaction_values": explainer.shap_interaction_values(background_data_tensor) if hasattr(explainer,  'shap_interaction_values') else None,
    "cumulative_shap_impact": np.sum(np.abs(shap_values_squeezed), axis=1),
    # "shap_feature_correlation": np.corrcoef(test_factors_np.T, shap_values_squeezed.T)[:d_model, d_model:],
    "shap_trend": np.mean(shap_values_squeezed, axis=1),
}


In [ ]:
shap_values[0].shape

In [ ]:
(train_returns - train_returns_mean) / train_returns_std

In [ ]:
train_returns_std

In [ ]:
returns_raw

In [ ]:
returns_raw_Y[[stock]]


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import ta
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Load data
factors = pd.read_csv("aligned_factors.csv", index_col=0, parse_dates=True)
returns = pd.read_csv("daily_returns_10ETFs.csv", index_col=0, parse_dates=True)

# Align dates
dates = factors.index.intersection(returns.index)
factors = factors.loc[dates]
returns = returns.loc[dates]

# Compute expanded generic technical indicators
technical_features = pd.DataFrame(index=returns.index)
for etf in returns.columns:
    close = (1 + returns[etf]).cumprod()
    technical_features[f'{etf}_SMA_5'] = ta.trend.sma_indicator(close, window=5)
    technical_features[f'{etf}_SMA_20'] = ta.trend.sma_indicator(close, window=20)
    technical_features[f'{etf}_SMA_50'] = ta.trend.sma_indicator(close, window=50)
    technical_features[f'{etf}_EMA_12'] = ta.trend.ema_indicator(close, window=12)
    technical_features[f'{etf}_EMA_26'] = ta.trend.ema_indicator(close, window=26)
    technical_features[f'{etf}_EMA_50'] = ta.trend.ema_indicator(close, window=50)
    technical_features[f'{etf}_RSI_7'] = ta.momentum.rsi(close, window=7)
    technical_features[f'{etf}_RSI_14'] = ta.momentum.rsi(close, window=14)
    technical_features[f'{etf}_MACD'] = ta.trend.macd_diff(close)
    technical_features[f'{etf}_ATR'] = ta.volatility.average_true_range(high=close*1.01, low=close*0.99, close=close, window=14)
    technical_features[f'{etf}_Volatility_5'] = returns[etf].rolling(window=5).std()
    technical_features[f'{etf}_Volatility_20'] = returns[etf].rolling(window=20).std()
    technical_features[f'{etf}_Volatility_50'] = returns[etf].rolling(window=50).std()
    technical_features[f'{etf}_Momentum_3'] = returns[etf].rolling(window=3).mean()
    technical_features[f'{etf}_Momentum_10'] = returns[etf].rolling(window=10).mean()

# Combine original factors with technical indicators
features = pd.concat([factors, technical_features], axis=1).dropna()

# Shift target by 1 day for next-day prediction
target_returns = returns.shift(-1).loc[features.index].dropna()
features = features.loc[target_returns.index]

# Define rolling window parameters
train_years = 10
valid_years = 2
start_year = 1999
end_year = 2024

# Identify expanded generic feature names
generic_features = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',
                    'SMA_5', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'EMA_50',
                    'RSI_7', 'RSI_14', 'MACD', 'ATR', 'Volatility_5', 'Volatility_20', 'Volatility_50',
                    'Momentum_3', 'Momentum_10']

# Step 1: Determine common generic important features
shap_importances_generic = pd.DataFrame(0, index=generic_features, columns=['SHAP_Value'])

for etf in returns.columns:
    print(f"Computing SHAP for generic feature selection ETF: {etf}")
    train_start = pd.Timestamp(2009 - train_years, 1, 1)
    train_end = pd.Timestamp(2009 - valid_years - 1, 12, 31)

    X_train = features.loc[train_start:train_end, [col for col in features.columns if etf in col or col in factors.columns]]
    y_train = target_returns[etf].loc[train_start:train_end]

    model = xgb.XGBRegressor(tree_method='hist', device='cuda').fit(X_train, y_train)
    explainer = shap.Explainer(model)
    shap_values = explainer(X_train)

    for generic in generic_features:
        cols = [col for col in X_train.columns if generic in col]
        shap_importances_generic.loc[generic] += np.mean(np.abs(shap_values.values[:, [X_train.columns.get_loc(c) for c in cols]]))

shap_importances_generic /= len(returns.columns)
common_generic_features = shap_importances_generic.sort_values('SHAP_Value', ascending=False).head(30).index.tolist()

In [ ]:
common_generic_features

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Step 2: Retrain models using generic common important features
shap_results = []
predictions_results = []

for etf in returns.columns[0:1]:
    print(f"Processing ETF: {etf}")
    etf_specific_features = [f for f in features.columns if any(generic in f for generic in common_generic_features) or f in factors.columns]

    for year in range(2009, 2010):
        print(f"Processing year: {year}")
        train_start = pd.Timestamp(year - train_years, 1, 1)
        train_end = pd.Timestamp(year - valid_years - 1, 12, 31)
        valid_start = pd.Timestamp(year - valid_years, 1, 1)
        valid_end = pd.Timestamp(year - 1, 12, 31)
        test_start = pd.Timestamp(year, 1, 1)
        test_end = pd.Timestamp(year, 12, 31)

        X_train = features.loc[train_start:train_end, etf_specific_features]
        y_train = target_returns[etf].loc[train_start:train_end]

        X_valid = features.loc[valid_start:valid_end, etf_specific_features]
        y_valid = target_returns[etf].loc[valid_start:valid_end]

        X_test = features.loc[test_start:test_end, etf_specific_features]
        y_test = target_returns[etf].loc[test_start:test_end]

        xgb_model = xgb.XGBRegressor(
            n_estimators=200,
            max_depth=4,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='reg:squarederror',
            random_state=42,
            tree_method='hist',
            device='cuda'
        )

        params = {'n_estimators': [100, 200], 'max_depth': [3, 4], 'learning_rate': [0.01, 0.05]}

        grid_search = GridSearchCV(xgb_model, params, cv=TimeSeriesSplit(n_splits=3), scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
        grid_search.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=True)

        best_model = grid_search.best_estimator_
        daily_preds = best_model.predict(X_test)
        
        y_true = y_test.values
        y_pred = daily_preds
        
        # Compute metrics
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        directional_accuracy = np.mean((np.sign(y_true) == np.sign(y_pred)).astype(int))
        
        print(f"MSE: {mse:.6f}")
        print(f"RMSE: {rmse:.6f}")
        print(f"MAE: {mae:.6f}")
        print(f"R-squared: {r2:.6f}")
        print(f"Directional Accuracy: {directional_accuracy:.2%}")

In [ ]:
common_generic_features[0:10]

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# After prediction loop, example for a single test period:
y_true = y_test.values
y_pred = daily_preds

# Compute metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
directional_accuracy = np.mean((np.sign(y_true) == np.sign(y_pred)).astype(int))

print(f"MSE: {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"R-squared: {r2:.6f}")
print(f"Directional Accuracy: {directional_accuracy:.2%}")

In [ ]:
# Save predictions
preds_df = pd.DataFrame({"Date": X_test.index, "ETF": etf, "Actual": y_test, "Predicted": daily_preds})
predictions_results.append(preds_df)

# Compute SHAP values
explainer = shap.Explainer(best_model)
shap_values = explainer(X_test)
shap_importance = np.abs(shap_values.values).mean(axis=0)
important_features = np.array(feature_cols)[shap_importance.argsort()[::-1][:30]]  # top 30 features
important_features
X_train_selected = X_train[important_features]
X_valid_selected = X_valid[important_features]
X_test_selected = X_test[important_features]

X_valid_selected = X_valid[important_features]

grid_search.fit(
    X_train_selected, y_train,
    eval_set=[(X_valid_selected, y_valid)],  # <-- explicitly include validation data
    verbose=True
)


In [ ]:
best_selected_model = grid_search.best_estimator_

# Make predictions explicitly using the selected features from X_test
predictions_selected = best_selected_model.predict(X_test_selected)

y_true = y_test.values
y_pred = predictions_selected = best_selected_model.predict(X_test_selected)


# Compute metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
directional_accuracy = np.mean((np.sign(y_true) == np.sign(y_pred)).astype(int))

print(f"MSE: {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"R-squared: {r2:.6f}")
print(f"Directional Accuracy: {directional_accuracy:.2%}")


In [ ]:

import pandas as pd
import numpy as np
import xgboost as xgb
import shap
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Load data
factors = pd.read_csv("aligned_factors.csv", index_col=0, parse_dates=True)
returns = pd.read_csv("daily_returns_10ETFs.csv", index_col=0, parse_dates=True)

# Align dates
dates = factors.index.intersection(returns.index)
factors = factors.loc[dates]
returns = returns.loc[dates]

# Shift target by 1 day for next-day prediction
target_returns = returns.shift(-1).dropna()
factors = factors.loc[target_returns.index]

# Shift target by 1 day for next-day prediction
target_returns = returns.shift(-1).dropna()
factors = factors.loc[target_returns.index]

# Define rolling window parameters
train_years = 10
valid_years = 2
start_year = 1999
end_year = 2024

# Placeholder for SHAP results
shap_results = []

# Feature and target columns
feature_cols = factors.columns
target_cols = target_returns.columns

print(f'target_cols={target_cols}')

In [ ]:
# stage 1 training and prediction 
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import ta
import joblib
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
# Load data
factors = pd.read_csv("aligned_factors.csv", index_col=0, parse_dates=True)
returns = pd.read_csv("daily_returns_10ETFs.csv", index_col=0, parse_dates=True)

# Align dates
dates = factors.index.intersection(returns.index)
factors = factors.loc[dates]
returns = returns.loc[dates]

# Compute expanded generic technical indicators
all_tech_features = []

for etf in returns.columns:
    close = (1 + returns[etf]).cumprod()
    etf_tech_features = pd.DataFrame(index=returns.index)

    etf_tech_features[f'{etf}_SMA_5'] = ta.trend.sma_indicator(close, window=5)
    etf_tech_features[f'{etf}_SMA_20'] = ta.trend.sma_indicator(close, window=20)
    etf_tech_features[f'{etf}_SMA_50'] = ta.trend.sma_indicator(close, window=50)
    etf_tech_features[f'{etf}_EMA_12'] = ta.trend.ema_indicator(close, window=12)
    etf_tech_features[f'{etf}_EMA_26'] = ta.trend.ema_indicator(close, window=26)
    etf_tech_features[f'{etf}_EMA_50'] = ta.trend.ema_indicator(close, window=50)
    etf_tech_features[f'{etf}_RSI_7'] = ta.momentum.rsi(close, window=7)
    etf_tech_features[f'{etf}_RSI_14'] = ta.momentum.rsi(close, window=14)
    etf_tech_features[f'{etf}_MACD'] = ta.trend.macd_diff(close)
    etf_tech_features[f'{etf}_ATR'] = ta.volatility.average_true_range(high=close*1.01, low=close*0.99, close=close, window=14)
    etf_tech_features[f'{etf}_Volatility_5'] = returns[etf].rolling(window=5).std()
    etf_tech_features[f'{etf}_Volatility_20'] = returns[etf].rolling(window=20).std()
    etf_tech_features[f'{etf}_Volatility_50'] = returns[etf].rolling(window=50).std()
    etf_tech_features[f'{etf}_Momentum_3'] = returns[etf].rolling(window=3).mean()
    etf_tech_features[f'{etf}_Momentum_10'] = returns[etf].rolling(window=10).mean()

    all_tech_features.append(etf_tech_features)

# Concatenate all ETF technical features at once to prevent DataFrame fragmentation
technical_features = pd.concat(all_tech_features, axis=1)

# Combine original factors with technical indicators
features = pd.concat([factors, technical_features], axis=1).dropna()

# Shift target by 1 day for next-day prediction
target_returns = returns.shift(-1).loc[features.index].dropna()
features = features.loc[target_returns.index]

# Define rolling window parameters
train_years = 10          # Length of training data in years
valid_years = 2           # Length of validation data in years
test_years = 1            # Length of testing/prediction data in years (configurable)
retrain_frequency = 1     # Retrain model every N years (configurable)
start_year = 2009
end_year = 2024

# Generic feature names
all_generic_features = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',
                        'SMA_5', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'EMA_50',
                        'RSI_7', 'RSI_14', 'MACD', 'ATR', 'Volatility_5', 'Volatility_20', 'Volatility_50',
                        'Momentum_3', 'Momentum_10']

# Step 1: Determine top N generic important features using aggregated SHAP across all ETFs
shap_importances = pd.DataFrame(0.0, index=all_generic_features, columns=['SHAP_Value'])

for etf in returns.columns:
    print(f"Computing SHAP for ETF: {etf}")
    train_start = pd.Timestamp(2009 - train_years, 1, 1)
    train_end = pd.Timestamp(2009 - valid_years - 1, 12, 31)
    
    etf_features = [col for col in features.columns if etf in col or col in factors.columns]
    X_train = features.loc[train_start:train_end, etf_features]
    y_train = target_returns[etf].loc[train_start:train_end]

    model = xgb.XGBRegressor(tree_method='hist', device='cuda').fit(X_train, y_train)
    explainer = shap.Explainer(model)
    shap_values = explainer(X_train)

    for generic in all_generic_features:
        cols = [col for col in X_train.columns if generic in col]
        if cols:
            idx = [X_train.columns.get_loc(c) for c in cols]
            shap_importances.loc[generic] += np.mean(np.abs(shap_values.values[:, idx]))

shap_importances /= len(returns.columns)
common_generic_features = shap_importances.sort_values('SHAP_Value', ascending=False).head(10).index.tolist()

# Step 2: Retrain models using selected top generic important features
all_predictions = []
for etf in returns.columns:  # Adjust this slice for all ETFs
    
    print(f"Processing ETF: {etf}")
    selected_features = [f for f in features.columns if any(generic in f for generic in common_generic_features) or f in factors.columns]

    # for year in range(2009, 2010):  # Adjust range for all years
    year = start_year
    while year <= end_year - test_years + 1:
        print(f"\nTraining window starting year: {year}")
        start_time = time.time()
        train_start = pd.Timestamp(year - train_years, 1, 1)
        train_end = pd.Timestamp(year - valid_years - 1, 12, 31)
        valid_start = pd.Timestamp(year - valid_years, 1, 1)
        valid_end = pd.Timestamp(year - 1, 12, 31)
        test_start = pd.Timestamp(year, 1, 1)
        test_end = pd.Timestamp(year + test_years - 1, 12, 31)

        X_train = features.loc[train_start:train_end, selected_features]
        y_train = target_returns[etf].loc[train_start:train_end]

        X_valid = features.loc[valid_start:valid_end, selected_features]
        y_valid = target_returns[etf].loc[valid_start:valid_end]

        X_test = features.loc[test_start:test_end, selected_features]
        y_test = target_returns[etf].loc[test_start:test_end]

        model = xgb.XGBRegressor(
            tree_method='hist',
            device='cuda',
            objective='reg:squarederror',
            random_state=42,
            n_jobs=4
        )

        params = {
            'n_estimators': [100, 200],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.01, 0.03, 0.05],
            'subsample': [0.7, 0.8],
            'colsample_bytree': [0.7, 0.8]
        }

        grid_search = GridSearchCV(model, params, cv=TimeSeriesSplit(3), scoring='neg_mean_squared_error', verbose=1, n_jobs=4)
        grid_search.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

        best_model = grid_search.best_estimator_
       
        # Convert test data explicitly to DMatrix for GPU
        dtest = xgb.DMatrix(X_test, enable_categorical=False)
        
        # Make predictions using the best_model explicitly
        preds = best_model.get_booster().predict(dtest)


        mse = mean_squared_error(y_test, preds)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, preds)
        r2 = r2_score(y_test, preds)
        directional_accuracy = np.mean((np.sign(y_test) == np.sign(preds)).astype(int))

        print(f"MSE: {mse:.6f}, RMSE: {rmse:.6f}, MAE: {mae:.6f}, R2: {r2:.6f}, Directional Accuracy: {directional_accuracy:.2%}")
        joblib.dump(best_model, f"best_model_{etf}_{year}.joblib")

        # Save daily predictions
        predictions_df = pd.DataFrame({'Date': X_test.index, 'ETF': etf, 'Year': year, 
                                       'Actual_Return': y_test, 'Predicted_Return': preds})

        # SHAP values explicitly aligned
        explainer_test = shap.Explainer(best_model)
        shap_values_test = explainer_test(X_test)
        
        # Create SHAP DataFrame explicitly with 'Date' to ensure correct merging
        shap_df = pd.DataFrame(
            shap_values_test.values,
            columns=[f'SHAP_{col}' for col in X_test.columns],
            index=X_test.index
        ).reset_index().rename(columns={'index': 'Date'})
        
        # Ensure predictions_df has 'Date' column explicitly for merging
        predictions_df = predictions_df.reset_index(drop=True)
        
        # Merge explicitly by 'Date' to align SHAP values correctly
        predictions_df = pd.merge(predictions_df, shap_df, on='Date', how='left')
        
        # Append explicitly for each ETF-year combination
        all_predictions.append(predictions_df)
        year += retrain_frequency
        end_time = time.time()
        print(f"Elapsed time: {end_time - start_time:.4f} seconds")

final_predictions_df = pd.concat(all_predictions).reset_index(drop=True)
final_predictions_df.to_csv("stage1_predictions_with_shap.csv", index=False)

print("Stage 1 completed and data saved for Stage 2.")


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import ta
import joblib
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
# Load data
factors = pd.read_csv("aligned_factors.csv", index_col=0, parse_dates=True)
returns = pd.read_csv("daily_returns_10ETFs.csv", index_col=0, parse_dates=True)

# Align dates
dates = factors.index.intersection(returns.index)
factors = factors.loc[dates]
returns = returns.loc[dates]

# Compute expanded generic technical indicators
all_tech_features = []

for etf in returns.columns:
    close = (1 + returns[etf]).cumprod()
    etf_tech_features = pd.DataFrame(index=returns.index)

    etf_tech_features[f'{etf}_SMA_5'] = ta.trend.sma_indicator(close, window=5)
    etf_tech_features[f'{etf}_SMA_20'] = ta.trend.sma_indicator(close, window=20)
    etf_tech_features[f'{etf}_SMA_50'] = ta.trend.sma_indicator(close, window=50)
    etf_tech_features[f'{etf}_EMA_12'] = ta.trend.ema_indicator(close, window=12)
    etf_tech_features[f'{etf}_EMA_26'] = ta.trend.ema_indicator(close, window=26)
    etf_tech_features[f'{etf}_EMA_50'] = ta.trend.ema_indicator(close, window=50)
    etf_tech_features[f'{etf}_RSI_7'] = ta.momentum.rsi(close, window=7)
    etf_tech_features[f'{etf}_RSI_14'] = ta.momentum.rsi(close, window=14)
    etf_tech_features[f'{etf}_MACD'] = ta.trend.macd_diff(close)
    etf_tech_features[f'{etf}_ATR'] = ta.volatility.average_true_range(high=close*1.01, low=close*0.99, close=close, window=14)
    etf_tech_features[f'{etf}_Volatility_5'] = returns[etf].rolling(window=5).std()
    etf_tech_features[f'{etf}_Volatility_20'] = returns[etf].rolling(window=20).std()
    etf_tech_features[f'{etf}_Volatility_50'] = returns[etf].rolling(window=50).std()
    etf_tech_features[f'{etf}_Momentum_3'] = returns[etf].rolling(window=3).mean()
    etf_tech_features[f'{etf}_Momentum_10'] = returns[etf].rolling(window=10).mean()

    all_tech_features.append(etf_tech_features)

# Concatenate all ETF technical features at once to prevent DataFrame fragmentation
technical_features = pd.concat(all_tech_features, axis=1)

# Combine original factors with technical indicators
features = pd.concat([factors, technical_features], axis=1).dropna()

# Shift target by 1 day for next-day prediction
target_returns = returns.shift(-1).loc[features.index].dropna()
features = features.loc[target_returns.index]

# Define rolling window parameters
train_years = 10          # Length of training data in years
valid_years = 2           # Length of validation data in years
test_years = 1            # Length of testing/prediction data in years (configurable)
retrain_frequency = 1     # Retrain model every N years (configurable)
start_year = 2009
end_year = 2024

# Generic feature names
all_generic_features = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA',
                        'SMA_5', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'EMA_50',
                        'RSI_7', 'RSI_14', 'MACD', 'ATR', 'Volatility_5', 'Volatility_20', 'Volatility_50',
                        'Momentum_3', 'Momentum_10']

# Step 1: Determine top N generic important features using aggregated SHAP across all ETFs
shap_importances = pd.DataFrame(0.0, index=all_generic_features, columns=['SHAP_Value'])

for etf in returns.columns:
    print(f"Computing SHAP for ETF: {etf}")
    train_start = pd.Timestamp(2009 - train_years, 1, 1)
    train_end = pd.Timestamp(2009 - valid_years - 1, 12, 31)
    
    etf_features = [col for col in features.columns if etf in col or col in factors.columns]
    X_train = features.loc[train_start:train_end, etf_features]
    y_train = target_returns[etf].loc[train_start:train_end]

    model = xgb.XGBRegressor(tree_method='hist', device='cuda').fit(X_train, y_train)
    explainer = shap.Explainer(model)
    shap_values = explainer(X_train)

    for generic in all_generic_features:
        cols = [col for col in X_train.columns if generic in col]
        if cols:
            idx = [X_train.columns.get_loc(c) for c in cols]
            shap_importances.loc[generic] += np.mean(np.abs(shap_values.values[:, idx]))

shap_importances /= len(returns.columns)
common_generic_features = shap_importances.sort_values('SHAP_Value', ascending=False).head(10).index.tolist()

In [ ]:
common_generic_features

In [ ]:

import pandas as pd
import numpy as np

# Load stage 1 data explicitly
stage1_df = pd.read_csv("stage1_predictions_with_shap.csv", parse_dates=['Date'])

etfs = stage1_df['ETF'].unique()

# Initialize DataFrame for daily aggregated data explicitly
dates = sorted(stage1_df['Date'].unique())
aggregated_data = pd.DataFrame({'Date': dates})

# ETF-specific predicted returns, actual returns, and volatility
for etf in etfs:
    etf_data = stage1_df[stage1_df['ETF'] == etf].set_index('Date').sort_index()

    aggregated_data[f'Predicted_Return_{etf}'] = aggregated_data['Date'].map(etf_data['Predicted_Return'])
    aggregated_data[f'Actual_Return_{etf}'] = aggregated_data['Date'].map(etf_data['Actual_Return'])

    # Explicit rolling 5-day volatility calculation
    volatility = etf_data['Actual_Return'].rolling(window=5).std()
    aggregated_data[f'Volatility_{etf}'] = aggregated_data['Date'].map(volatility)

# Define explicitly generic SHAP features to aggregate across ETFs
# generic_shap_features = [
#     'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 
#     'SMA_5', 'SMA_20', 'SMA_50',
#     'EMA_12', 'EMA_26', 'EMA_50',
#     'RSI_7', 'RSI_14', 'MACD', 'ATR',
#     'Volatility_5', 'Volatility_20', 'Volatility_50',
#     'Momentum_3', 'Momentum_10'
# ]
generic_shap_features = ['Mkt-RF',
 'Volatility_50',
 'HML',
 'Momentum_3',
 'Volatility_5',
 'Volatility_20',
 'RMW',
 'SMB',
 'CMA',
 'RSI_7']

# Aggregate SHAP values explicitly across ETFs by generic feature
shap_aggregated = {'Date': dates}
shap_df_list = []

for feature in generic_shap_features:
    # Adjust matching explicitly for 'SHAP_{ETF}_{feature}' format
    matching_shap_cols = [col for col in stage1_df.columns if col.startswith('SHAP_') and col.endswith(f'_{feature}')]
    
    if matching_shap_cols:
        # Compute daily mean explicitly across selected SHAP columns
        daily_shap_mean = stage1_df.groupby('Date')[matching_shap_cols].mean().mean(axis=1)
        shap_df_list.append(daily_shap_mean.rename(f'Avg_SHAP_{feature}'))
    else:
        print(f"Warning: No matches found explicitly for feature: {feature}")

# Concatenate aggregated SHAP features explicitly, ensuring alignment
shap_aggregated_df = pd.concat(shap_df_list, axis=1).reset_index()

# Explicit merge with ETF-specific metrics on Date to ensure alignment
aggregated_data = pd.merge(aggregated_data, shap_aggregated_df, on='Date', how='left')

# Explicit handling of missing values
aggregated_data.sort_values('Date', inplace=True)
aggregated_data.fillna(method='ffill', inplace=True)

# Remove rows explicitly if initial volatility calculations have NaNs
vol_cols = [f'Volatility_{etf}' for etf in etfs]
aggregated_data.dropna(subset=vol_cols, inplace=True)

# Check if aggregated_data is empty before saving explicitly
if aggregated_data.empty:
    print("Warning: aggregated_data is empty after processing. Please verify input data and alignment explicitly.")
else:
    aggregated_data.to_csv("stage2_rl_observations_optimized.csv", index=False)
    print(f"Optimized Stage 2 RL dataset created with shape: {aggregated_data.shape}")


In [ ]:
# start of stage 2 training
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.model_selection import ParameterGrid

import gymnasium as gym
from gymnasium import spaces
import numpy as np
import time
class PortfolioEnv(gym.Env):
    def __init__(self, data, etf_list, reward_type='mean_cvar', risk_coefficient=0.5, rebalance_period=21, lookback_period=21):
        super().__init__()

        self.data = data.reset_index(drop=True)
        self.etf_list = etf_list
        self.reward_type = reward_type
        self.risk_coefficient = risk_coefficient
        self.rebalance_period = rebalance_period
        self.lookback_period = lookback_period
        self.action_space = spaces.Box(low=-1, high=1, shape=(len(etf_list),), dtype=np.float32)

        # Explicitly select feature columns (excluding Date and returns used only for calculating reward)
        self.feature_cols = [col for col in data.columns if col not in ['Date'] and not col.startswith('Actual_Return')]
        self.num_features_per_day = len(self.feature_cols)

        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.num_features_per_day * self.lookback_period,),
            dtype=np.float32
        )

        self.current_step = self.lookback_period
        self.done = False
        self.cumulative_wealth = 1.0
        self.current_weights = np.array([1.0 / len(etf_list)] * len(etf_list))

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.seed(seed)
        self.current_step = self.lookback_period
        self.done = False
        self.cumulative_wealth = 1.0
        self.current_weights = np.array([1.0 / len(self.etf_list)] * len(self.etf_list))
        return self._get_obs(), {}

    def step(self, action):
        next_step = self.current_step + 1

        if self.current_step % self.rebalance_period == 0:
            # v2 long short
            desired_long = 1.20  # 120% long exposure explicitly
            desired_short = 0.20  # 20% short exposure explicitly
            clip_bounds = (-0.2, 0.8)

            raw_weights = action.copy()

            # Separate explicitly positive (long) and negative (short) actions
            long_weights = np.maximum(raw_weights, 0)
            short_weights = np.abs(np.minimum(raw_weights, 0))

            has_longs = np.sum(long_weights) > 0
            has_shorts = np.sum(short_weights) > 0

            if has_longs and has_shorts:
                # Normal 120/20 explicitly0
                normalized_long = desired_long * long_weights / np.sum(long_weights)
                normalized_short = desired_short * short_weights / np.sum(short_weights)
            elif has_longs and not has_shorts:
                # Only long explicitly: default realistically to 100% long
                normalized_long = long_weights / np.sum(long_weights)
                normalized_short = np.zeros_like(short_weights)
            elif not has_longs and has_shorts:
                # Only short explicitly (unrealistic), fallback clearly to equal-weight long-only
                num_assets = len(raw_weights)
                normalized_long = np.ones(num_assets) / num_assets
                normalized_short = np.zeros(num_assets)
            else:
                # All zeros explicitly: fallback explicitly to equal-weight long-only
                num_assets = len(raw_weights)
                normalized_long = np.ones(num_assets) / num_assets
                normalized_short = np.zeros(num_assets)

            # Apply explicit clipping
            combined_weights = normalized_long - normalized_short
            clipped_weights = np.clip(combined_weights, clip_bounds[0], clip_bounds[1])

            # Re-separate explicitly after clipping
            long_clipped = np.maximum(clipped_weights, 0)
            short_clipped = np.abs(np.minimum(clipped_weights, 0))

            has_long_clipped = np.sum(long_clipped) > 0
            has_short_clipped = np.sum(short_clipped) > 0

            # Final explicit normalization after clipping
            if has_long_clipped and has_short_clipped:
                final_long = desired_long * long_clipped / np.sum(long_clipped)
                final_short = desired_short * short_clipped / np.sum(short_clipped)
            elif has_long_clipped and not has_short_clipped:
                final_long = long_clipped / np.sum(long_clipped)  # exactly 100% long
                final_short = np.zeros_like(short_clipped)
            else:
                # Realistic fallback explicitly: equal-weight long-only
                num_assets = len(raw_weights)
                final_long = np.ones(num_assets) / num_assets
                final_short = np.zeros(num_assets)

            final_weights = final_long - final_short
            self.current_weights = final_weights
            # v1 softmax normalization
            # 
            # temperature = 0.5  # Explicitly lower for higher concentration (try 0.2 to 0.8)
            # scaled_action = action / temperature
            # self.current_weights = np.exp(scaled_action) / np.sum(np.exp(scaled_action))

        else:
            returns_today = np.array([self.data.loc[self.current_step, f'Actual_Return_{etf}'] for etf in self.etf_list])
            self.current_weights *= (1 + returns_today)
            self.current_weights /= np.sum(self.current_weights)

        if next_step >= len(self.data):
            terminated = True
            reward = 0.0
        else:
            returns = np.array([self.data.loc[next_step, f'Actual_Return_{etf}'] for etf in self.etf_list])
            portfolio_return = np.dot(self.current_weights, returns)
            self.cumulative_wealth *= (1 + portfolio_return)
            reward = self.calculate_reward(portfolio_return, returns)
            terminated = next_step >= len(self.data) - 1

        self.current_step += 1

        return self._get_obs(), reward, terminated, False, {}

        # def _get_obs(self):
        #     obs_window = self.data.iloc[self.current_step - self.lookback_period:self.current_step]
        #     obs_window = obs_window.drop(columns=['Date']).values.flatten().astype(np.float32)
        #     return obs_window

    def _get_obs(self):
        obs_window = self.data.iloc[self.current_step - self.lookback_period:self.current_step]
        obs_window = obs_window[self.feature_cols].values.flatten().astype(np.float32)
        return obs_window

    def calculate_reward(self, portfolio_return, asset_returns):
        if self.reward_type == 'cumulative_return':
            return self.cumulative_wealth - 1.0
        elif self.reward_type == 'log_wealth':
            return np.log(self.cumulative_wealth)
        elif self.reward_type == 'mean_var':
            return portfolio_return - self.risk_coefficient * np.var(asset_returns)
        elif self.reward_type == 'mean_cvar':
            alpha = 0.05
            var = np.percentile(asset_returns, 100 * alpha)
            cvar = np.mean(asset_returns[asset_returns <= var])
            return portfolio_return - self.risk_coefficient * cvar
        else:
            raise ValueError('Invalid reward type')

    def seed(self, seed=None):
        np.random.seed(seed)

import pandas as pd
import numpy as np

def add_stable_features(df, etf_list):
    data = df.copy()

    for etf in etf_list:
        price_col = f'Price_{etf}'

        # Volatility (20-day)
        data[f'Volatility_{etf}'] = data[price_col].pct_change().rolling(20).std()

        # Momentum indicators (returns over 5, 10, 20 days)
        data[f'Momentum_5d_{etf}'] = data[price_col].pct_change(periods=5)
        data[f'Momentum_10d_{etf}'] = data[price_col].pct_change(periods=10)
        data[f'Momentum_20d_{etf}'] = data[price_col].pct_change(periods=20)

        # Moving averages (5-day and 20-day)
        data[f'MA_5d_{etf}'] = data[price_col].rolling(5).mean()
        data[f'MA_20d_{etf}'] = data[price_col].rolling(20).mean()

        # Moving average crossover (5-day MA - 20-day MA)
        data[f'MA_Crossover_{etf}'] = data[f'MA_5d_{etf}'] - data[f'MA_20d_{etf}']

    # Drop NaN values due to rolling calculations
    data.dropna(inplace=True)

    return data

def filter_features(df, include_predicted_returns=True, include_shap_metrics=True):
    df_filtered = df.copy()

    # Explicit patterns to identify columns
    predicted_return_pattern = 'Predicted_Return'
    shap_metric_pattern = 'SHAP'

    # Exclude Predicted Returns explicitly if requested
    if not include_predicted_returns:
        predicted_cols = [col for col in df_filtered.columns if predicted_return_pattern in col]
        df_filtered.drop(columns=predicted_cols, inplace=True)
        print(f"Excluded predicted return columns: {predicted_cols}")

    # Exclude SHAP-related metrics explicitly if requested
    if not include_shap_metrics:
        shap_cols = [col for col in df_filtered.columns if shap_metric_pattern in col]
        df_filtered.drop(columns=shap_cols, inplace=True)
        print(f"Excluded SHAP-related columns: {shap_cols}")

    return df_filtered

# ETFs
etf_list = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLY', 'XLV', 'XLU']

# Hyperparameter tuning
param_grid = {
    'learning_rate': [1e-4, 5e-5],
    'n_steps': [20, 40],
    'batch_size': [5, 10],
    'gamma': [0.98, 0.99]
}
consolidated_file = 'stage2_rl_observations_optimized.csv'
reward_type = 'log_wealth'
# data = pd.read_csv(consolidated_file, parse_dates=['Date'])
# data = data.sort_values('Date').reset_index(drop=True)

data = pd.read_csv('stage2_rl_observations_optimized.csv', parse_dates=['Date'])
price_data = pd.read_csv('stock_prices_10ETFs.csv')

# Convert the Date column in price data, handling the timezone correctly
price_data['Date'] = pd.to_datetime(price_data['Date'], utc=True)
price_data['Date'] = price_data['Date'].dt.tz_localize(None)

# Rename price columns explicitly to 'price_{ticker}'
price_cols = {col: f'Price_{col}' for col in price_data.columns if col != 'Date'}
price_data.rename(columns=price_cols, inplace=True)

# Merge datasets on Date
merged_data = pd.merge(data, price_data, on='Date', how='inner')
merged_data.reset_index(drop=True, inplace=True)
# Check if merge was successful
if len(merged_data) != len(data):
    print(f"Warning: Data length mismatch after merging (Original: {len(data)}, Merged: {len(merged_data)}).")
else:
    print("Merged successfully with aligned dates.")

data_with_features_raw = add_stable_features(merged_data, etf_list)
data_with_features_raw.reset_index(drop=True, inplace=True)


# Usage Example clearly for benchmark (only price metrics, no predicted return or SHAP):
data_with_features = filter_features(data_with_features_raw, 
                                 include_predicted_returns=True, 
                                 include_shap_metrics=True)
################### override data to use SHAP only
# data_with_features = data
################### END override 

# Define your rolling window lengths clearly:
train_window_days = 252 * 7
validation_window_days = 252
prediction_window_days = 252
lookback_period = 21
rebalance_period = 21

start_indices = range(0, len(data) - (train_window_days + validation_window_days + prediction_window_days), prediction_window_days)
all_weights = []

from sklearn.model_selection import ParameterSampler
def validate_and_tune(train_data, val_data, reward_type, rebalance_period=10, lookback_period=10, n_iter=8, timesteps=5000):
    best_reward, best_params = -np.inf, None

    # Narrow and meaningful parameter distribution
    param_dist = {
        'learning_rate': [3e-4, 1e-4],
        'n_steps': [20, 40],
        'batch_size': [10, 20],
        'gamma': [0.95, 0.98],
        'risk_coefficient': [0.1, 0.5, 1.0] if reward_type in ['mean_var', 'mean_cvar'] else [0.5],
    }

    sampled_params = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=42))

    for params in sampled_params:
        risk_coeff = params.pop('risk_coefficient', 0.5)

        env = make_vec_env(lambda: PortfolioEnv(train_data, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period), n_envs=1)
        model = PPO('MlpPolicy', env,
                    ent_coef=0.01,    # explicitly encourages exploration
                    clip_range=0.2,
                    **params, verbose=0)
        model.learn(total_timesteps=timesteps)

        val_env = PortfolioEnv(val_data, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)
        obs, _ = val_env.reset()
        done, total_reward = False, 0

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _, _ = val_env.step(action)
            total_reward += reward

        if total_reward > best_reward:
            best_reward = total_reward
            best_params = {**params, 'risk_coefficient': risk_coeff}

    return best_params

def scale_data(df, feature_cols, scaler):
    scaled_features = scaler.transform(df[feature_cols])
    scaled_df = pd.DataFrame(scaled_features, columns=feature_cols, index=df.index)

    # Re-add columns that were not scaled (e.g., Date, Actual_Return_*)
    for col in df.columns:
        if col not in feature_cols:
            scaled_df[col] = df[col].values

    # Keep original column order
    scaled_df = scaled_df[df.columns]
    return scaled_df

# Main execution
from sklearn.preprocessing import StandardScaler
for start_idx in start_indices:
    # for start_idx in range(0, 252*2, 252):
    start_time = time.time()

    # Explicit indices for training, validation, and prediction datasets
    train_start_idx = start_idx
    train_end_idx = train_start_idx + train_window_days

    val_start_idx = train_end_idx
    val_end_idx = val_start_idx + validation_window_days

    pred_start_idx = val_end_idx
    pred_end_idx = pred_start_idx + prediction_window_days

    # Corresponding dates explicitly
    train_start_date = data_with_features.loc[train_start_idx, 'Date']
    train_end_date = data_with_features.loc[train_end_idx - 1, 'Date']

    val_start_date = data_with_features.loc[val_start_idx, 'Date']
    val_end_date = data_with_features.loc[val_end_idx - 1, 'Date']

    pred_start_date = data_with_features.loc[pred_start_idx, 'Date']
    pred_end_date = data_with_features.loc[pred_end_idx - 1, 'Date']

    # Clearly print ranges for clarity
    print(f"Training period: {train_start_date.date()} to {train_end_date.date()}")
    print(f"Validation period: {val_start_date.date()} to {val_end_date.date()}")
    print(f"Prediction period: {pred_start_date.date()} to {pred_end_date.date()}")

    # Explicitly subset data accordingly
    train_data = data_with_features.iloc[train_start_idx:train_end_idx].reset_index(drop=True)
    val_data = data_with_features.iloc[val_start_idx:val_end_idx].reset_index(drop=True)
    pred_data = data_with_features.iloc[pred_start_idx:pred_end_idx].reset_index(drop=True)

    feature_cols = [col for col in train_data.columns if col != 'Date' and not col.startswith('Actual_Return')]

    scaler = StandardScaler()
    scaler.fit(train_data[feature_cols])

    train_data_scaled = scale_data(train_data, feature_cols, scaler)
    val_data_scaled = scale_data(val_data, feature_cols, scaler)
    pred_data_scaled = scale_data(pred_data, feature_cols, scaler)

    print("Starting hyperparameter tuning...")
    best_params = validate_and_tune(train_data_scaled, val_data_scaled, reward_type)
    print(f"Best parameters: {best_params}")

    incremental_timesteps = 20000
    max_timesteps = 100000
    patience = 3
    
    best_val_reward = -np.inf
    no_improve_steps = 0

    risk_coeff = best_params.pop('risk_coefficient',0.5)
    policy_kwargs = dict(net_arch=[256, 256])

    env = make_vec_env(lambda: PortfolioEnv(train_data_scaled, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period), n_envs=1)
    model = PPO('MlpPolicy', env,
                policy_kwargs=policy_kwargs,
                ent_coef=0.01,    # explicitly encourages exploration
                clip_range=0.2,
                **best_params, verbose=0)
    # print("Starting model training...")
    # model.learn(total_timesteps=20000)
    print("Starting model training with early stopping...")

    for step in range(0, max_timesteps, incremental_timesteps):
        model.learn(total_timesteps=incremental_timesteps)
    
        # Evaluate on validation environment
        val_env = PortfolioEnv(val_data_scaled, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)
        val_obs, _ = val_env.reset()
        val_done = False
        val_total_reward = 0.0
    
        while not val_done:
            val_action, _ = model.predict(val_obs, deterministic=True)
            val_obs, val_reward, val_done, _, _ = val_env.step(val_action)
            val_total_reward += val_reward
    
        print(f"Step: {step + incremental_timesteps}, Validation Total Reward: {val_total_reward:.4f}")
    
        # Early stopping check
        if val_total_reward > best_val_reward:
            best_val_reward = val_total_reward
            no_improve_steps = 0
            model.save("best_ppo_model.zip")
            print(f"Improved validation reward; model saved at step {step + incremental_timesteps}")
        else:
            no_improve_steps += 1
            print(f"No improvement ({no_improve_steps}/{patience})")
    
            if no_improve_steps >= patience:
                print("Early stopping explicitly triggered.")
                break
    
    # Load the best model explicitly
    model = PPO.load("best_ppo_model.zip")
    print("Loaded the best PPO model explicitly for prediction.")



    # Ensure historical context explicitly available in prediction
    full_data = pd.concat([train_data_scaled, val_data_scaled, pred_data_scaled])
    pred_data_with_history = full_data[full_data['Date'] >= (pred_start_date - pd.Timedelta(days=lookback_period))].reset_index(drop=True)

    pred_env = PortfolioEnv(pred_data_scaled, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)
    # pred_env = PortfolioEnv(pred_data_with_history, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)

    obs, info = pred_env.reset()
    done = False

    action = np.zeros(len(etf_list), dtype=np.float32)

    while not done:
        if pred_env.current_step >= lookback_period and pred_env.current_step % pred_env.rebalance_period == 0:
            # obs_for_agent = pred_data_with_history.drop(columns=['Date']).iloc[pred_env.current_step - lookback_period:pred_env.current_step].values.flatten().astype(np.float32)
            # action, _ = model.predict(obs_for_agent, deterministic=True)

            # v1 normalize weight
            # action, _ = model.predict(obs, deterministic=True)
            # temperature = 0.5
            # scaled_action = action / temperature
            # weights = np.exp(scaled_action) / np.sum(np.exp(scaled_action))
            # rebalance_date = pred_data_with_history.loc[pred_env.current_step, 'Date']
            # all_weights.append([rebalance_date] + weights.tolist())


            # v2 long short normalization
            action, _ = model.predict(obs, deterministic=True)

            # Explicitly apply your new 120/20 normalization logic (to match environment step)
            desired_long = 1.20  # Explicitly 120% long exposure
            desired_short = 0.20  # Explicitly 20% short exposure
            clip_bounds = (-0.2, 0.8)

            raw_weights = action.copy()

            # Separate explicitly positive (long) and negative (short) actions
            long_weights = np.maximum(raw_weights, 0)
            short_weights = np.abs(np.minimum(raw_weights, 0))

            has_longs = np.sum(long_weights) > 0
            has_shorts = np.sum(short_weights) > 0

            if has_longs and has_shorts:
                normalized_long = desired_long * long_weights / np.sum(long_weights)
                normalized_short = desired_short * short_weights / np.sum(short_weights)
            elif has_longs and not has_shorts:
                normalized_long = long_weights / np.sum(long_weights)
                normalized_short = np.zeros_like(short_weights)
            elif not has_longs and has_shorts:
                num_assets = len(raw_weights)
                normalized_long = np.ones(num_assets) / num_assets
                normalized_short = np.zeros(num_assets)
            else:
                num_assets = len(raw_weights)
                normalized_long = np.ones(num_assets) / num_assets
                normalized_short = np.zeros(num_assets)

            combined_weights = normalized_long - normalized_short
            clipped_weights = np.clip(combined_weights, clip_bounds[0], clip_bounds[1])

            # Re-separate after clipping explicitly
            long_clipped = np.maximum(clipped_weights, 0)
            short_clipped = np.abs(np.minimum(clipped_weights, 0))

            has_long_clipped = np.sum(long_clipped) > 0
            has_short_clipped = np.sum(short_clipped) > 0

            if has_long_clipped and has_short_clipped:
                final_long = desired_long * long_clipped / np.sum(long_clipped)
                final_short = desired_short * short_clipped / np.sum(short_clipped)
            elif has_long_clipped and not has_short_clipped:
                final_long = long_clipped / np.sum(long_clipped)
                final_short = np.zeros_like(short_clipped)
            else:
                num_assets = len(raw_weights)
                final_long = np.ones(num_assets) / num_assets
                final_short = np.zeros(num_assets)

            final_weights = final_long - final_short

            rebalance_date = pred_data_with_history.loc[pred_env.current_step, 'Date']
            all_weights.append([rebalance_date] + final_weights.tolist())

        obs, _, done, _, _ = pred_env.step(action)

    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time:.4f} seconds")

columns = ['Date'] + etf_list
weights_df = pd.DataFrame(all_weights, columns=columns)
weights_df.to_csv('ppo_multi_year_weights.csv', index=False)
print("Saved predictions to ppo_multi_year_weights.csv")


In [10]:
# ITERATION
# start of stage 2 training
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from sklearn.model_selection import ParameterGrid

import gymnasium as gym
from gymnasium import spaces
import numpy as np
import time
class PortfolioEnv(gym.Env):
    def __init__(self, data, etf_list, reward_type='mean_cvar', risk_coefficient=0.5, rebalance_period=21, lookback_period=21):
        super().__init__()

        self.data = data.reset_index(drop=True)
        self.etf_list = etf_list
        self.reward_type = reward_type
        self.risk_coefficient = risk_coefficient
        self.rebalance_period = rebalance_period
        self.lookback_period = lookback_period
        self.action_space = spaces.Box(low=-1, high=1, shape=(len(etf_list),), dtype=np.float32)

        # Explicitly select feature columns (excluding Date and returns used only for calculating reward)
        self.feature_cols = [col for col in data.columns if col not in ['Date'] and not col.startswith('Actual_Return')]
        self.num_features_per_day = len(self.feature_cols)

        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.num_features_per_day * self.lookback_period,),
            dtype=np.float32
        )

        self.current_step = self.lookback_period
        self.done = False
        self.cumulative_wealth = 1.0
        self.current_weights = np.array([1.0 / len(etf_list)] * len(etf_list))

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.seed(seed)
        self.current_step = self.lookback_period
        self.done = False
        self.cumulative_wealth = 1.0
        self.current_weights = np.array([1.0 / len(self.etf_list)] * len(self.etf_list))
        return self._get_obs(), {}

    def step(self, action):
        next_step = self.current_step + 1

        if self.current_step % self.rebalance_period == 0:
            # v2 long short
            desired_long = 1.20  # 120% long exposure explicitly
            desired_short = 0.20  # 20% short exposure explicitly
            clip_bounds = (-0.2, 0.8)

            raw_weights = action.copy()

            # Separate explicitly positive (long) and negative (short) actions
            long_weights = np.maximum(raw_weights, 0)
            short_weights = np.abs(np.minimum(raw_weights, 0))

            has_longs = np.sum(long_weights) > 0
            has_shorts = np.sum(short_weights) > 0

            if has_longs and has_shorts:
                # Normal 120/20 explicitly0
                normalized_long = desired_long * long_weights / np.sum(long_weights)
                normalized_short = desired_short * short_weights / np.sum(short_weights)
            elif has_longs and not has_shorts:
                # Only long explicitly: default realistically to 100% long
                normalized_long = long_weights / np.sum(long_weights)
                normalized_short = np.zeros_like(short_weights)
            elif not has_longs and has_shorts:
                # Only short explicitly (unrealistic), fallback clearly to equal-weight long-only
                num_assets = len(raw_weights)
                normalized_long = np.ones(num_assets) / num_assets
                normalized_short = np.zeros(num_assets)
            else:
                # All zeros explicitly: fallback explicitly to equal-weight long-only
                num_assets = len(raw_weights)
                normalized_long = np.ones(num_assets) / num_assets
                normalized_short = np.zeros(num_assets)

            # Apply explicit clipping
            combined_weights = normalized_long - normalized_short
            clipped_weights = np.clip(combined_weights, clip_bounds[0], clip_bounds[1])

            # Re-separate explicitly after clipping
            long_clipped = np.maximum(clipped_weights, 0)
            short_clipped = np.abs(np.minimum(clipped_weights, 0))

            has_long_clipped = np.sum(long_clipped) > 0
            has_short_clipped = np.sum(short_clipped) > 0

            # Final explicit normalization after clipping
            if has_long_clipped and has_short_clipped:
                final_long = desired_long * long_clipped / np.sum(long_clipped)
                final_short = desired_short * short_clipped / np.sum(short_clipped)
            elif has_long_clipped and not has_short_clipped:
                final_long = long_clipped / np.sum(long_clipped)  # exactly 100% long
                final_short = np.zeros_like(short_clipped)
            else:
                # Realistic fallback explicitly: equal-weight long-only
                num_assets = len(raw_weights)
                final_long = np.ones(num_assets) / num_assets
                final_short = np.zeros(num_assets)

            final_weights = final_long - final_short
            self.current_weights = final_weights
            # v1 softmax normalization
            # 
            # temperature = 0.5  # Explicitly lower for higher concentration (try 0.2 to 0.8)
            # scaled_action = action / temperature
            # self.current_weights = np.exp(scaled_action) / np.sum(np.exp(scaled_action))

        else:
            returns_today = np.array([self.data.loc[self.current_step, f'Actual_Return_{etf}'] for etf in self.etf_list])
            self.current_weights *= (1 + returns_today)
            self.current_weights /= np.sum(self.current_weights)

        if next_step >= len(self.data):
            terminated = True
            reward = 0.0
        else:
            returns = np.array([self.data.loc[next_step, f'Actual_Return_{etf}'] for etf in self.etf_list])
            portfolio_return = np.dot(self.current_weights, returns)
            self.cumulative_wealth *= (1 + portfolio_return)
            reward = self.calculate_reward(portfolio_return, returns)
            terminated = next_step >= len(self.data) - 1

        self.current_step += 1

        return self._get_obs(), reward, terminated, False, {}

        # def _get_obs(self):
        #     obs_window = self.data.iloc[self.current_step - self.lookback_period:self.current_step]
        #     obs_window = obs_window.drop(columns=['Date']).values.flatten().astype(np.float32)
        #     return obs_window

    def _get_obs(self):
        obs_window = self.data.iloc[self.current_step - self.lookback_period:self.current_step]
        obs_window = obs_window[self.feature_cols].values.flatten().astype(np.float32)
        return obs_window

    def calculate_reward(self, portfolio_return, asset_returns):
        if self.reward_type == 'cumulative_return':
            return self.cumulative_wealth - 1.0
        elif self.reward_type == 'log_wealth':
            return np.log(self.cumulative_wealth)
        elif self.reward_type == 'mean_var':
            return portfolio_return - self.risk_coefficient * np.var(asset_returns)
        elif self.reward_type == 'mean_cvar':
            alpha = 0.05
            var = np.percentile(asset_returns, 100 * alpha)
            cvar = np.mean(asset_returns[asset_returns <= var])
            return portfolio_return - self.risk_coefficient * cvar
        else:
            raise ValueError('Invalid reward type')

    def seed(self, seed=None):
        np.random.seed(seed)

import pandas as pd
import numpy as np

def add_stable_features(df, etf_list):
    data = df.copy()

    for etf in etf_list:
        price_col = f'Price_{etf}'

        # Volatility (20-day)
        data[f'Volatility_{etf}'] = data[price_col].pct_change().rolling(20).std()

        # Momentum indicators (returns over 5, 10, 20 days)
        data[f'Momentum_5d_{etf}'] = data[price_col].pct_change(periods=5)
        data[f'Momentum_10d_{etf}'] = data[price_col].pct_change(periods=10)
        data[f'Momentum_20d_{etf}'] = data[price_col].pct_change(periods=20)

        # Moving averages (5-day and 20-day)
        data[f'MA_5d_{etf}'] = data[price_col].rolling(5).mean()
        data[f'MA_20d_{etf}'] = data[price_col].rolling(20).mean()

        # Moving average crossover (5-day MA - 20-day MA)
        data[f'MA_Crossover_{etf}'] = data[f'MA_5d_{etf}'] - data[f'MA_20d_{etf}']

    # Drop NaN values due to rolling calculations
    data.dropna(inplace=True)

    return data

def filter_features(df, include_predicted_returns=True, include_shap_metrics=True):
    df_filtered = df.copy()

    # Explicit patterns to identify columns
    predicted_return_pattern = 'Predicted_Return'
    shap_metric_pattern = 'SHAP'

    # Exclude Predicted Returns explicitly if requested
    if not include_predicted_returns:
        predicted_cols = [col for col in df_filtered.columns if predicted_return_pattern in col]
        df_filtered.drop(columns=predicted_cols, inplace=True)
        print(f"Excluded predicted return columns: {predicted_cols}")

    # Exclude SHAP-related metrics explicitly if requested
    if not include_shap_metrics:
        shap_cols = [col for col in df_filtered.columns if shap_metric_pattern in col]
        df_filtered.drop(columns=shap_cols, inplace=True)
        print(f"Excluded SHAP-related columns: {shap_cols}")

    return df_filtered

# ETFs
etf_list = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLY', 'XLV', 'XLU']

# Hyperparameter tuning
param_grid = {
    'learning_rate': [1e-4, 5e-5],
    'n_steps': [20, 40],
    'batch_size': [5, 10],
    'gamma': [0.98, 0.99]
}
consolidated_file = 'stage2_rl_observations_optimized.csv'
reward_type = 'log_wealth'
# data = pd.read_csv(consolidated_file, parse_dates=['Date'])
# data = data.sort_values('Date').reset_index(drop=True)

data = pd.read_csv('stage2_rl_observations_optimized.csv', parse_dates=['Date'])
price_data = pd.read_csv('stock_prices_10ETFs.csv')

# Convert the Date column in price data, handling the timezone correctly
price_data['Date'] = pd.to_datetime(price_data['Date'], utc=True)
price_data['Date'] = price_data['Date'].dt.tz_localize(None)

# Rename price columns explicitly to 'price_{ticker}'
price_cols = {col: f'Price_{col}' for col in price_data.columns if col != 'Date'}
price_data.rename(columns=price_cols, inplace=True)

# Merge datasets on Date
merged_data = pd.merge(data, price_data, on='Date', how='inner')
merged_data.reset_index(drop=True, inplace=True)
# Check if merge was successful
if len(merged_data) != len(data):
    print(f"Warning: Data length mismatch after merging (Original: {len(data)}, Merged: {len(merged_data)}).")
else:
    print("Merged successfully with aligned dates.")

data_with_features_raw = add_stable_features(merged_data, etf_list)
data_with_features_raw.reset_index(drop=True, inplace=True)


# Usage Example clearly for benchmark (only price metrics, no predicted return or SHAP):
data_with_features = filter_features(data_with_features_raw, 
                                 include_predicted_returns=True, 
                                 include_shap_metrics=True)
################### override data to use SHAP only
# data_with_features = data
################### END override 

# Define your rolling window lengths clearly:
train_window_days = 252 * 7
validation_window_days = 252
prediction_window_days = 252
lookback_period = 21
rebalance_period = 21

start_indices = range(0, len(data) - (train_window_days + validation_window_days + prediction_window_days), prediction_window_days)
all_weights = []

from sklearn.model_selection import ParameterSampler
def validate_and_tune(train_data, val_data, reward_type, rebalance_period=10, lookback_period=10, n_iter=8, timesteps=5000):
    best_reward, best_params = -np.inf, None

    # Narrow and meaningful parameter distribution
    param_dist = {
        'learning_rate': [3e-4, 1e-4],
        'n_steps': [20, 40],
        'batch_size': [10, 20],
        'gamma': [0.95, 0.98],
        'risk_coefficient': [0.1, 0.5, 1.0] if reward_type in ['mean_var', 'mean_cvar'] else [0.5],
    }

    sampled_params = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=42))

    for params in sampled_params:
        risk_coeff = params.pop('risk_coefficient', 0.5)

        env = make_vec_env(lambda: PortfolioEnv(train_data, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period), n_envs=1)
        model = PPO('MlpPolicy', env,
                    ent_coef=0.01,    # explicitly encourages exploration
                    clip_range=0.2,
                    **params, verbose=0)
        model.learn(total_timesteps=timesteps)

        val_env = PortfolioEnv(val_data, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)
        obs, _ = val_env.reset()
        done, total_reward = False, 0

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _, _ = val_env.step(action)
            total_reward += reward

        if total_reward > best_reward:
            best_reward = total_reward
            best_params = {**params, 'risk_coefficient': risk_coeff}

    return best_params

def scale_data(df, feature_cols, scaler):
    scaled_features = scaler.transform(df[feature_cols])
    scaled_df = pd.DataFrame(scaled_features, columns=feature_cols, index=df.index)

    # Re-add columns that were not scaled (e.g., Date, Actual_Return_*)
    for col in df.columns:
        if col not in feature_cols:
            scaled_df[col] = df[col].values

    # Keep original column order
    scaled_df = scaled_df[df.columns]
    return scaled_df

# Main execution
from sklearn.preprocessing import StandardScaler

iterations = 25
all_weights_iterations = []

for iteration in range(iterations):
    print(f"\n==== Starting Iteration {iteration + 1}/{iterations} ====")

    for start_idx in start_indices:
        # for start_idx in range(0, 252*2, 252):
        start_time = time.time()
    
        # Explicit indices for training, validation, and prediction datasets
        train_start_idx = start_idx
        train_end_idx = train_start_idx + train_window_days
    
        val_start_idx = train_end_idx
        val_end_idx = val_start_idx + validation_window_days
    
        pred_start_idx = val_end_idx
        pred_end_idx = pred_start_idx + prediction_window_days
    
        # Corresponding dates explicitly
        train_start_date = data_with_features.loc[train_start_idx, 'Date']
        train_end_date = data_with_features.loc[train_end_idx - 1, 'Date']
    
        val_start_date = data_with_features.loc[val_start_idx, 'Date']
        val_end_date = data_with_features.loc[val_end_idx - 1, 'Date']
    
        pred_start_date = data_with_features.loc[pred_start_idx, 'Date']
        pred_end_date = data_with_features.loc[pred_end_idx - 1, 'Date']
    
        # Clearly print ranges for clarity
        print(f"Training period: {train_start_date.date()} to {train_end_date.date()}")
        print(f"Validation period: {val_start_date.date()} to {val_end_date.date()}")
        print(f"Prediction period: {pred_start_date.date()} to {pred_end_date.date()}")
    
        # Explicitly subset data accordingly
        train_data = data_with_features.iloc[train_start_idx:train_end_idx].reset_index(drop=True)
        val_data = data_with_features.iloc[val_start_idx:val_end_idx].reset_index(drop=True)
        pred_data = data_with_features.iloc[pred_start_idx:pred_end_idx].reset_index(drop=True)
    
        feature_cols = [col for col in train_data.columns if col != 'Date' and not col.startswith('Actual_Return')]
    
        scaler = StandardScaler()
        scaler.fit(train_data[feature_cols])
    
        train_data_scaled = scale_data(train_data, feature_cols, scaler)
        val_data_scaled = scale_data(val_data, feature_cols, scaler)
        pred_data_scaled = scale_data(pred_data, feature_cols, scaler)
    
        print("Starting hyperparameter tuning...")
        best_params = validate_and_tune(train_data_scaled, val_data_scaled, reward_type)
        print(f"Best parameters: {best_params}")
    
        incremental_timesteps = 20000
        max_timesteps = 100000
        patience = 3
        
        best_val_reward = -np.inf
        no_improve_steps = 0
    
        risk_coeff = best_params.pop('risk_coefficient',0.5)
        policy_kwargs = dict(net_arch=[256, 256])
    
        env = make_vec_env(lambda: PortfolioEnv(train_data_scaled, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period), n_envs=1)
        model = PPO('MlpPolicy', env,
                    policy_kwargs=policy_kwargs,
                    ent_coef=0.01,    # explicitly encourages exploration
                    clip_range=0.2,
                    **best_params, verbose=0)
        # print("Starting model training...")
        # model.learn(total_timesteps=20000)
        print("Starting model training with early stopping...")
    
        for step in range(0, max_timesteps, incremental_timesteps):
            model.learn(total_timesteps=incremental_timesteps)
        
            # Evaluate on validation environment
            val_env = PortfolioEnv(val_data_scaled, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)
            val_obs, _ = val_env.reset()
            val_done = False
            val_total_reward = 0.0
        
            while not val_done:
                val_action, _ = model.predict(val_obs, deterministic=True)
                val_obs, val_reward, val_done, _, _ = val_env.step(val_action)
                val_total_reward += val_reward
        
            print(f"Step: {step + incremental_timesteps}, Validation Total Reward: {val_total_reward:.4f}")
        
            # Early stopping check
            if val_total_reward > best_val_reward:
                best_val_reward = val_total_reward
                no_improve_steps = 0
                model.save("best_ppo_model.zip")
                print(f"Improved validation reward; model saved at step {step + incremental_timesteps}")
            else:
                no_improve_steps += 1
                print(f"No improvement ({no_improve_steps}/{patience})")
        
                if no_improve_steps >= patience:
                    print("Early stopping explicitly triggered.")
                    break
        
        # Load the best model explicitly
        model = PPO.load("best_ppo_model.zip")
        print("Loaded the best PPO model explicitly for prediction.")
    
    
    
        # Ensure historical context explicitly available in prediction
        full_data = pd.concat([train_data_scaled, val_data_scaled, pred_data_scaled])
        pred_data_with_history = full_data[full_data['Date'] >= (pred_start_date - pd.Timedelta(days=lookback_period))].reset_index(drop=True)
    
        pred_env = PortfolioEnv(pred_data_scaled, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)
        # pred_env = PortfolioEnv(pred_data_with_history, etf_list, reward_type, risk_coeff, rebalance_period, lookback_period)
    
        obs, info = pred_env.reset()
        done = False
    
        action = np.zeros(len(etf_list), dtype=np.float32)
    
        while not done:
            if pred_env.current_step >= lookback_period and pred_env.current_step % pred_env.rebalance_period == 0:
                # obs_for_agent = pred_data_with_history.drop(columns=['Date']).iloc[pred_env.current_step - lookback_period:pred_env.current_step].values.flatten().astype(np.float32)
                # action, _ = model.predict(obs_for_agent, deterministic=True)
    
                # v1 normalize weight
                # action, _ = model.predict(obs, deterministic=True)
                # temperature = 0.5
                # scaled_action = action / temperature
                # weights = np.exp(scaled_action) / np.sum(np.exp(scaled_action))
                # rebalance_date = pred_data_with_history.loc[pred_env.current_step, 'Date']
                # all_weights.append([rebalance_date] + weights.tolist())
    
    
                # v2 long short normalization
                action, _ = model.predict(obs, deterministic=True)
    
                # Explicitly apply your new 120/20 normalization logic (to match environment step)
                desired_long = 1.20  # Explicitly 120% long exposure
                desired_short = 0.20  # Explicitly 20% short exposure
                clip_bounds = (-0.2, 0.8)
    
                raw_weights = action.copy()
    
                # Separate explicitly positive (long) and negative (short) actions
                long_weights = np.maximum(raw_weights, 0)
                short_weights = np.abs(np.minimum(raw_weights, 0))
    
                has_longs = np.sum(long_weights) > 0
                has_shorts = np.sum(short_weights) > 0
    
                if has_longs and has_shorts:
                    normalized_long = desired_long * long_weights / np.sum(long_weights)
                    normalized_short = desired_short * short_weights / np.sum(short_weights)
                elif has_longs and not has_shorts:
                    normalized_long = long_weights / np.sum(long_weights)
                    normalized_short = np.zeros_like(short_weights)
                elif not has_longs and has_shorts:
                    num_assets = len(raw_weights)
                    normalized_long = np.ones(num_assets) / num_assets
                    normalized_short = np.zeros(num_assets)
                else:
                    num_assets = len(raw_weights)
                    normalized_long = np.ones(num_assets) / num_assets
                    normalized_short = np.zeros(num_assets)
    
                combined_weights = normalized_long - normalized_short
                clipped_weights = np.clip(combined_weights, clip_bounds[0], clip_bounds[1])
    
                # Re-separate after clipping explicitly
                long_clipped = np.maximum(clipped_weights, 0)
                short_clipped = np.abs(np.minimum(clipped_weights, 0))
    
                has_long_clipped = np.sum(long_clipped) > 0
                has_short_clipped = np.sum(short_clipped) > 0
    
                if has_long_clipped and has_short_clipped:
                    final_long = desired_long * long_clipped / np.sum(long_clipped)
                    final_short = desired_short * short_clipped / np.sum(short_clipped)
                elif has_long_clipped and not has_short_clipped:
                    final_long = long_clipped / np.sum(long_clipped)
                    final_short = np.zeros_like(short_clipped)
                else:
                    num_assets = len(raw_weights)
                    final_long = np.ones(num_assets) / num_assets
                    final_short = np.zeros(num_assets)
    
                final_weights = final_long - final_short
    
                rebalance_date = pred_data_with_history.loc[pred_env.current_step, 'Date']
                # all_weights.append([rebalance_date] + final_weights.tolist())
                all_weights_iterations.append([iteration + 1, rebalance_date] + final_weights.tolist())
            obs, _, done, _, _ = pred_env.step(action)
    
        end_time = time.time()
        print(f"Iteration {iteration + 1}, start index {start_idx} completed in {end_time - start_time:.4f} seconds")

columns = ['Iteration', 'Date'] + etf_list
weights_df = pd.DataFrame(all_weights_iterations, columns=columns)
weights_df.to_csv('ppo_allocations_multiple_iterations.csv', index=False)
print("Saved all iterations' allocations to ppo_allocations_multiple_iterations.csv")



Merged successfully with aligned dates.

==== Starting Iteration 1/25 ====
Training period: 2009-02-06 to 2016-02-09
Validation period: 2016-02-10 to 2017-02-08
Prediction period: 2017-02-09 to 2018-02-08
Starting hyperparameter tuning...
Best parameters: {'n_steps': 40, 'learning_rate': 0.0001, 'gamma': 0.95, 'batch_size': 20, 'risk_coefficient': 0.5}
Starting model training with early stopping...
Step: 20000, Validation Total Reward: 6.1312
Improved validation reward; model saved at step 20000
Step: 40000, Validation Total Reward: 16.7238
Improved validation reward; model saved at step 40000
Step: 60000, Validation Total Reward: 19.0038
Improved validation reward; model saved at step 60000
Step: 80000, Validation Total Reward: 20.0287
Improved validation reward; model saved at step 80000
Step: 100000, Validation Total Reward: 18.5619
No improvement (1/3)
Loaded the best PPO model explicitly for prediction.
Iteration 1, start index 0 completed in 671.9926 seconds
Training period: 2010

In [ ]:
train_data_scaled

In [ ]:
def filter_features(df, include_predicted_returns=True, include_shap_metrics=True):
    df_filtered = df.copy()

    # Explicit patterns to identify columns
    predicted_return_pattern = 'Predicted_Return'
    shap_metric_pattern = 'SHAP'

    # Exclude Predicted Returns explicitly if requested
    if not include_predicted_returns:
        predicted_cols = [col for col in df_filtered.columns if predicted_return_pattern in col]
        df_filtered.drop(columns=predicted_cols, inplace=True)
        print(f"Excluded predicted return columns: {predicted_cols}")

    # Exclude SHAP-related metrics explicitly if requested
    if not include_shap_metrics:
        shap_cols = [col for col in df_filtered.columns if shap_metric_pattern in col]
        df_filtered.drop(columns=shap_cols, inplace=True)
        print(f"Excluded SHAP-related columns: {shap_cols}")

    return df_filtered

# Usage Example clearly for benchmark (only price metrics, no predicted return or SHAP):
benchmark_data = filter_features(data_with_features, 
                                 include_predicted_returns=False, 
                                 include_shap_metrics=False)

print("Benchmark data shape:", benchmark_data.shape)

In [ ]:
data_with_features.to_csv('test.csv')

In [ ]:
columns = ['Date'] + etf_list
weights_df = pd.DataFrame(all_weights, columns=columns)
weights_df.to_csv('ppo_multi_year_weights.csv', index=False)
print("Saved predictions to ppo_multi_year_weights.csv")


In [ ]:
merged_data.columns


In [ ]:
# end of the stage 2 portfolio allocation 

In [ ]:
!pip install --upgrade numba

In [ ]:
train_factors_np = train_factors_norm.values.astype('float32')
num_samples, d_model = train_factors_np.shape  # Extract number of samples and feature count
print(f'num_samples: {num_samples}, d_model:{d_model}')
background_data = train_factors_norm.values.astype('float32').reshape(-1, 126, d_model)
background_data_tensor = torch.tensor(background_data, dtype=torch.float32).to(device)

# ✅ Debugging SHAP input shape
print(
    f"✅ Background Data Shape Before SHAP: {background_data_tensor.shape}")  # Expected: (1, rolling_window_test, 6)

# ✅ Step 3: Initialize SHAP GradientExplainer using training data
explainer = shap.GradientExplainer(model, background_data_tensor)

# ✅ Step 4: Compute SHAP values for the training set
shap_values = explainer.shap_values(torch.tensor(background_data, dtype=torch.float32).to(device))
print(f'shap_values: {shap_values}')

# ✅ Step 5: Debug SHAP value shape before squeezing
print(f"🚀 SHAP Raw Output Shape: {shap_values[0].shape}")  # Example: (1, rolling_window_test, 6, 1)


In [ ]:


# +-------------------------------+           +-------------------------------+
# | DIA SHAP + Technical Indicators|           |    Daily Returns (28 Stocks)  |
# | (from shap_with_DIA_indicators.csv)        |  (from daily_returns.csv)     |
# +--------------+----------------+           +---------------+---------------+
#                |                                            |
#                v                                            v
#         +---------------------+                  +--------------------------+
#         |  High-Level Agent   |----------------->|   Low-Level PPO Agents   |
#         |  (PPO classifier)   | Regime Signal    | (trained per market regime)|
#         +---------------------+                  +--------------------------+
#                |                                            |
#                v                                            v
#         Regime prediction                         Optimal portfolio weights
#                                                   (Mean-CVaR reward explicitly benchmarked vs naive)

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import gym
from gym import spaces
import torch
import os

# === PPO Environments ===
class HighLevelEnv(gym.Env):
    def __init__(self, X, regimes):
        super().__init__()
        self.X = X
        self.regimes = regimes
        self.index = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X.shape[1],))
        self.action_space = spaces.Discrete(3)

    def reset(self):
        self.index = 0
        return self.X[self.index]

    def step(self, action):
        reward = 1 if action == self.regimes[self.index] else -1
        self.index += 1
        done = self.index >= len(self.X)-1
        obs = self.X[self.index] if not done else np.zeros(self.X.shape[1])
        return obs, reward, done, {}

class LowLevelBenchmarkEnv(gym.Env):
    def __init__(self, returns, window_size=21, lambda_risk=0.5, alpha=0.05):
        super().__init__()
        self.returns = returns
        self.window_size = window_size
        self.lambda_risk = lambda_risk
        self.alpha = alpha
        self.n_assets = returns.shape[1]
        self.index = window_size
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(window_size, self.n_assets))
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,))

    def reset(self):
        self.index = self.window_size
        return self.returns[self.index-self.window_size:self.index]

    def step(self, action):
        action = np.clip(action, 0, 1)
        action /= np.sum(action + 1e-8)

        # Agent and naive returns
        agent_return = np.dot(action, self.returns[self.index])
        naive_weights = np.array([1/self.n_assets] * self.n_assets)
        naive_return = np.dot(naive_weights, self.returns[self.index])

        # Excess return explicitly benchmarking naive portfolio
        excess_return = agent_return - naive_return

        historical_returns = self.returns[self.index-self.window_size:self.index] @ action
        cvar_threshold = np.percentile(historical_returns, 100*self.alpha)
        cvar_penalty = max(cvar_threshold - agent_return, 0)

        # Reward explicitly includes naive portfolio benchmark
        reward = excess_return - self.lambda_risk * cvar_penalty

        self.index += 1
        done = self.index >= len(self.returns)-1
        obs = self.returns[self.index-self.window_size:self.index] if not done else np.zeros((self.window_size, self.n_assets))

        return obs, reward, done, {
            "agent_return": agent_return,
            "naive_return": naive_return,
            "excess_return": excess_return,
            "cvar_penalty": cvar_penalty
        }




In [ ]:
# start from here
# start from here
# start from here
import pandas as pd
import numpy as np
import ta

# === Load daily returns ===
returns_df = pd.read_csv("daily_returns.csv", parse_dates=["Date"])
returns_df.set_index("Date", inplace=True)

# Convert daily returns explicitly into price series (starting at 100)
price_df = (1 + returns_df).cumprod() * 100

# Define function explicitly to generate indicators per stock
def generate_stock_features(df_prices):
    df = pd.DataFrame(index=df_prices.index)
    df['Price'] = df_prices
    
    # Daily returns explicitly
    df['Return_1d'] = df['Price'].pct_change()
    df['Return_5d'] = df['Price'].pct_change(5)
    df['Return_21d'] = df['Price'].pct_change(21)
    
    # Volatility explicitly calculated
    df['Volatility_5d'] = df['Return_1d'].rolling(window=5).std()
    df['Volatility_21d'] = df['Return_1d'].rolling(window=21).std()
    
    # Drawdown explicitly calculated
    df['Drawdown'] = df['Price'] / df['Price'].cummax() - 1
    
    # Technical indicators explicitly calculated
    df['SMA_20'] = ta.trend.sma_indicator(df['Price'], window=20)
    df['SMA_50'] = ta.trend.sma_indicator(df['Price'], window=50)
    df['EMA_12'] = ta.trend.ema_indicator(df['Price'], window=12)
    df['EMA_26'] = ta.trend.ema_indicator(df['Price'], window=26)
    df['MACD'] = ta.trend.macd_diff(df['Price'])
    df['RSI_14'] = ta.momentum.rsi(df['Price'], window=14)
    
    bb_indicator = ta.volatility.BollingerBands(df['Price'], window=20)
    df['BB_High'] = bb_indicator.bollinger_hband()
    df['BB_Low'] = bb_indicator.bollinger_lband()
    
    atr_indicator = ta.volatility.AverageTrueRange(
        high=df['Price'] * 1.01,
        low=df['Price'] * 0.99,
        close=df['Price'],
        window=14
    )
    df['ATR_14'] = atr_indicator.average_true_range()
    
    # Drop NaNs explicitly (initial periods with insufficient data)
    df = df.dropna()
    
    return df

# Generate features explicitly for all 28 stocks
all_features = {}
for ticker in price_df.columns:
    print(f"Generating factors explicitly for {ticker}...")
    stock_prices = price_df[ticker]
    stock_features = generate_stock_features(stock_prices)
    stock_features.columns = [f"{ticker}_{col}" for col in stock_features.columns]
    all_features[ticker] = stock_features

# Combine explicitly into single DataFrame
features_df = pd.concat(all_features.values(), axis=1).dropna()

# Save explicitly to CSV
features_df.to_csv("stock_factors_28_stocks.csv")
print("✅ Explicit factors generated and saved as 'stock_factors_28_stocks.csv'.")


In [ ]:
# class HighLevelEnv(gym.Env):
#     def __init__(self, X, regimes, returns):
#         self.X = X.astype(np.float32)
#         self.regimes = regimes
#         self.returns = returns
#         self.index = 0
#         self.observation_space = spaces.Box(-np.inf, np.inf, shape=(X.shape[1],), dtype=np.float32)
#         self.action_space = spaces.Discrete(3)  # explicit matching n_components
# 
#     def reset(self, seed=None, options=None):
#         self.index = 0
#         return self.X[self.index], {}
# 
#     def step(self, action):
#         classification_reward = 1 if action == self.regimes[self.index] else -1
#         financial_reward = np.mean(self.returns[self.index:self.index+predict_window])
#         reward = 0.7 * classification_reward + 0.3 * financial_reward
#         self.index += 1
#         terminated = self.index >= len(self.X) - predict_window - 1
#         obs = self.X[self.index] if not terminated else np.zeros(self.X.shape[1], dtype=np.float32)
#         return obs, reward, terminated, False, {}

class HighLevelEnv(gym.Env):
    def __init__(self, X, regimes, consistency_window=5):
        self.X = X.astype(np.float32)
        self.regimes = regimes
        self.index = 0
        self.consistency_window = consistency_window
        self.prev_actions = []
        
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(X.shape[1],), dtype=np.float32)
        self.action_space = spaces.Discrete(3)

    def reset(self, seed=None, options=None):
        self.index = 0
        self.prev_actions = []
        return self.X[self.index], {}

    def step(self, action):
        action = int(action) 
        classification_reward = 1 if action == self.regimes[self.index] else -1
        
        # Append the current action
        self.prev_actions.append(action)
        
        # Compute consistency reward
        consistency_reward = 0
        if len(self.prev_actions) >= self.consistency_window:
            recent_actions = self.prev_actions[-self.consistency_window:]
            recent_regimes = self.regimes[self.index - self.consistency_window + 1 : self.index + 1]
            
            # Check consistency (all correct)
            if np.array_equal(recent_actions, recent_regimes):
                consistency_reward = 0.5  # bonus for consistent accuracy
        
        reward = classification_reward + consistency_reward
        
        self.index += 1
        terminated = self.index >= len(self.X) - 1
        obs = self.X[self.index] if not terminated else np.zeros(self.X.shape[1], dtype=np.float32)
        
        return obs, reward, terminated, False, {}
    
# class LowLevelPortfolioEnv(gym.Env):
#     def __init__(self, features, returns, regimes, window_size=21, predict_window=5, lambda_risk=0.5, alpha=0.05):
#         self.features = features.astype(np.float32)
#         self.returns = returns.astype(np.float32)
#         self.regimes = regimes
#         self.window_size = window_size
#         self.predict_window = predict_window
#         self.lambda_risk = lambda_risk
#         self.alpha = alpha
#         self.n_assets = returns.shape[1]
#         self.index = window_size
# 
#         obs_dim = window_size * features.shape[1] + 3
#         self.observation_space = spaces.Box(-np.inf, np.inf, shape=(obs_dim,), dtype=np.float32)
#         self.action_space = spaces.Box(low=-1, high=1, shape=(self.n_assets,), dtype=np.float32)
#         self.regime_encoding = {0: [1,0,0], 1: [0,1,0], 2: [0,0,1]}
# 
#     def reset(self, seed=None, options=None):
#         super().reset(seed=seed)
#         self.index = self.window_size
#         return self._get_obs(), {}
# 
#     def _get_obs(self):
#         recent_features = self.features[self.index-self.window_size:self.index].flatten()
#         regime_onehot = self.regime_encoding[self.regimes[self.index - 1]]  # Fixed indexing explicitly
#         return np.concatenate([recent_features, regime_onehot]).astype(np.float32)
# 
#     def step(self, action):
#         action = (action + 1) / 2
#         action = np.clip(action, 0, 1)
#         action /= np.sum(action + 1e-8)
# 
#         start_idx = self.index
#         end_idx = min(self.index + self.predict_window, len(self.returns))
# 
#         next_period_returns = self.returns[start_idx:end_idx]
#         port_returns = next_period_returns @ action
# 
#         # Use cumulative return explicitly as the reward
#         cumulative_return = np.prod(1 + port_returns) - 1
# 
#         # Metrics explicitly retained for additional analysis
#         naive_returns = np.mean(next_period_returns, axis=1)
#         cumulative_naive_return = np.prod(1 + naive_returns) - 1
#         excess_return = cumulative_return - cumulative_naive_return
#         cvar_threshold = np.percentile(port_returns, self.alpha * 100)
#         mean_cvar_reward = np.mean(port_returns) - self.lambda_risk * max(cvar_threshold - np.mean(port_returns), 0)
# 
#         reward = excess_return  # explicitly changed here
# 
#         self.index += self.predict_window
#         terminated = self.index >= len(self.returns) - self.predict_window
# 
#         obs = self._get_obs() if not terminated else np.zeros(self.observation_space.shape, dtype=np.float32)
# 
#         info = {
#             "cumulative_return": cumulative_return,
#             "mean_cvar_reward": mean_cvar_reward,
#             "excess_return": excess_return
#         }
# 
#         return obs, float(reward), terminated, False, info
# incoporate Shap in ENV
class LowLevelPortfolioEnv(gym.Env):
    def __init__(self, features, returns, shap_features, regimes, window_size=21, predict_window=5, lambda_risk=0.5, alpha=0.05):
        self.features = features.astype(np.float32)
        self.shap_features = shap_features.astype(np.float32)
        self.returns = returns.astype(np.float32)
        self.regimes = regimes
        self.window_size = window_size
        self.predict_window = predict_window
        self.lambda_risk = lambda_risk
        self.alpha = alpha
        self.n_assets = returns.shape[1]
        self.index = window_size

        obs_dim = (window_size * features.shape[1]) + shap_features.shape[1] + 3
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(obs_dim,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.n_assets,), dtype=np.float32)
        self.regime_encoding = {0: [1,0,0], 1: [0,1,0], 2: [0,0,1]}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.index = self.window_size
        return self._get_obs(), {}

    def _get_obs(self):
        recent_features = self.features[self.index - self.window_size:self.index].flatten()
        current_shap_features = self.shap_features[self.index - 1]  # explicitly using the latest SHAP/DIA features
        regime_onehot = self.regime_encoding[self.regimes[self.index - 1]]
        return np.concatenate([recent_features, current_shap_features, regime_onehot]).astype(np.float32)

    def step(self, action):
        action = (action + 1) / 2
        action = np.clip(action, 0, 1)
        action /= np.sum(action + 1e-8)

        start_idx = self.index
        end_idx = min(self.index + self.predict_window, len(self.returns))

        next_period_returns = self.returns[start_idx:end_idx]
        port_returns = next_period_returns @ action

        cumulative_return = np.prod(1 + port_returns) - 1
        reward = cumulative_return

        self.index += self.predict_window
        terminated = self.index >= len(self.returns) - self.predict_window

        obs = self._get_obs() if not terminated else np.zeros(self.observation_space.shape, dtype=np.float32)

        info = {"cumulative_return": cumulative_return}

        return obs, float(reward), terminated, False, info


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
import os

# === Load Data Explicitly ===
shap_df = pd.read_csv("shap_with_DIA_indicators.csv", parse_dates=["End_Date"])
returns_df = pd.read_csv("daily_returns_DIA.csv", parse_dates=["Date"]).set_index("Date")

data = pd.merge(shap_df, returns_df, left_on="End_Date", right_index=True).dropna()

features = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA', 'mean_abs_shap_Stock',
    'mean_shap_Mkt-RF', 'mean_shap_SMB', 'mean_shap_HML', 'mean_shap_RMW',
    'mean_shap_CMA', 'mean_shap_Stock', 'shap_std_Mkt-RF', 'shap_std_SMB',
    'shap_std_HML', 'shap_std_RMW', 'shap_std_CMA', 'shap_std_Stock',
    'mean_over_std_Mkt-RF', 'mean_over_std_SMB', 'mean_over_std_HML',
    'mean_over_std_RMW', 'mean_over_std_CMA', 'mean_over_std_Stock',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB',
    'mean_abs_over_std_HML', 'mean_abs_over_std_RMW',
    'mean_abs_over_std_CMA', 'mean_abs_over_std_Stock',
    'Return_1d', 'Return_5d', 'Rolling_Return_21d',
    'Volatility_5d', 'Volatility_21d', 'Drawdown',
    'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'RSI_14'
]

# Parameters
train_window, predict_window, step_size, consistency_window = 360, 5, 5, 5
model_dir = "high_level_saved_models"
os.makedirs(model_dir, exist_ok=True)
retrain = False
# Explicit PPO training function
def train_or_load_agent(env, model_path, total_timesteps, continue_training=False):
    if continue_training and os.path.exists(model_path):
        print(f"✅ Loading existing model from {model_path}")
        agent = PPO.load(model_path, env=env)
    else:
        print("✅ Initializing new PPO agent.")
        agent = PPO("MlpPolicy", env, verbose=0)

    print(f"✅ Training for {total_timesteps} timesteps.")
    agent.learn(total_timesteps=total_timesteps)

    print(f"✅ Saving model explicitly to {model_path}")
    agent.save(model_path)

    return agent

regime_results, accuracy_log = [], []
# Rolling-window explicitly defined
for start in range(0, len(data) - train_window - predict_window, step_size):
# for start in range(0, 400, step_size):
    start_time = time.time()

    train_data = data.iloc[start:start + train_window].copy()
    predict_data = data.iloc[start + train_window:start + train_window + predict_window].copy()

    X_train_hl = train_data[features].values
    X_predict_hl = predict_data[features].values

    scaler_hl = StandardScaler()
    X_train_hl_scaled = scaler_hl.fit_transform(X_train_hl)
    X_predict_hl_scaled = scaler_hl.transform(X_predict_hl)

    gmm = GaussianMixture(n_components=3, random_state=42)
    regimes_train = gmm.fit_predict(X_train_hl_scaled)

    regimes_train_next = np.roll(regimes_train, -step_size)
    X_train_aligned = X_train_hl_scaled[:-step_size]
    regimes_train_aligned = regimes_train_next[:-step_size]

    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train_aligned, regimes_train_aligned)
    supervised_pred = clf.predict(X_predict_hl_scaled)

    env_high = HighLevelEnv(X_train_hl_scaled, regimes_train_next, consistency_window)

    model_path = os.path.join(model_dir, f"ppo_model_step_{start}.zip")
    initial_iterations = 2000

    high_level_agent = train_or_load_agent(
        env=env_high,
        model_path=model_path,
        total_timesteps=initial_iterations,
        # continue_training=os.path.exists(model_path)
        continue_training=retrain
        
    )

    predicted_regimes = [int(high_level_agent.predict(obs, deterministic=True)[0]) for obs in X_predict_hl_scaled]

    final_predicted_regimes = [
        ppo_pred if ppo_pred == sup_pred else sup_pred
        for ppo_pred, sup_pred in zip(predicted_regimes, supervised_pred)
    ]

    regimes_predict_actual = gmm.predict(X_predict_hl_scaled)
    accuracy = np.mean(final_predicted_regimes == regimes_predict_actual)
    accuracy_log.append(accuracy)

    regime_results.append(pd.DataFrame({
        'Date': predict_data['End_Date'].values,
        'Predicted_Regime': final_predicted_regimes,
        'Actual_Regime': regimes_predict_actual
    }))

    print(f"✅ Step {start}: Trained from {train_data['End_Date'].iloc[0].date()} "
          f"to {train_data['End_Date'].iloc[-1].date()}, "
          f"predicted starting {predict_data['End_Date'].iloc[0].date()}, Accuracy: {accuracy:.2%}")

    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time:.4f} seconds")

final_results = pd.concat(regime_results, ignore_index=True)
final_results.to_csv("high_level_rolling_predictions_with_actual.csv", index=False)

accuracy_overall = np.mean(accuracy_log)
print(f"✅ Overall accuracy: {accuracy_overall:.2%}")
print("Results saved to 'high_level_rolling_predictions_with_actual.csv'")



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from stable_baselines3 import PPO
import gymnasium as gym
from gymnasium import spaces
from xgboost import XGBClassifier
import os
import time

# Load Data Explicitly
shap_df = pd.read_csv("shap_with_DIA_indicators.csv", parse_dates=["End_Date"])
returns_df = pd.read_csv("daily_returns_DIA.csv", parse_dates=["Date"]).set_index("Date")
data = pd.merge(shap_df, returns_df, left_on="End_Date", right_index=True).dropna()

features = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA', 'mean_abs_shap_Stock',
    'mean_shap_Mkt-RF', 'mean_shap_SMB', 'mean_shap_HML', 'mean_shap_RMW',
    'mean_shap_CMA', 'mean_shap_Stock', 'shap_std_Mkt-RF', 'shap_std_SMB',
    'shap_std_HML', 'shap_std_RMW', 'shap_std_CMA', 'shap_std_Stock',
    'Return_1d', 'Return_5d', 'Rolling_Return_21d',
    'Volatility_5d', 'Volatility_21d', 'Drawdown',
    'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'RSI_14'
]

train_window, predict_window, step_size = 360, 10, 10
model_dir = "high_level_saved_models"
os.makedirs(model_dir, exist_ok=True)

class HighLevelEnv(gym.Env):
    def __init__(self, X, next_regime, consistency_window=5):
        self.X = X
        self.next_regime = next_regime
        self.consistency_window = consistency_window
        self.index = 0
        self.prev_actions = []

        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(X.shape[1],), dtype=np.float32)
        self.action_space = spaces.Discrete(3)

    def reset(self, seed=None, options=None):
        self.index = 0
        self.prev_actions = []
        return self.X[self.index], {}

    def step(self, action):
        reward = 1 if action == self.next_regime[self.index] else -1

        self.prev_actions.append(action)
        if len(self.prev_actions) >= self.consistency_window:
            if np.all(self.prev_actions[-self.consistency_window:] == 
                      self.next_regime[self.index - self.consistency_window + 1: self.index + 1]):
                reward += 0.5

        self.index += 1
        terminated = self.index >= len(self.X) - 1
        obs = self.X[self.index] if not terminated else np.zeros(self.X.shape[1], dtype=np.float32)
        return obs, reward, terminated, False, {}

regime_results, accuracy_log = [], []

# Define absolute thresholds explicitly
negative_threshold, positive_threshold = -0.001, 0.001  # Customize these thresholds as needed

for start in range(0, len(data) - train_window - predict_window, step_size):
    start_time = time.time()

    train_data = data.iloc[start:start + train_window].copy()
    predict_data = data.iloc[start + train_window:start + train_window + predict_window].copy()

    scaler = StandardScaler()
    X_train = scaler.fit_transform(train_data[features].values)
    X_predict = scaler.transform(predict_data[features].values)
    
    # Explicit regime definition (absolute returns)
    regime_features = train_data[['Return_1d']].values.flatten()
    regimes_current = np.where(regime_features < negative_threshold, 0,
                        np.where(regime_features < positive_threshold, 1, 2))
    
    # Shifted target explicitly for future prediction
    regimes_next_period = np.roll(regimes_current, -predict_window)
    regimes_next_period_aligned = regimes_next_period[:-predict_window]
    X_train_aligned = X_train[:-predict_window]
    
    env_high = HighLevelEnv(X_train_aligned, regimes_next_period_aligned)
    model_path = os.path.join(model_dir, f"ppo_model_step_{start}.zip")

    agent = PPO(
        "MlpPolicy",
        env_high,
        learning_rate=1e-4,
        gamma=0.98,
        n_steps=1024,
        ent_coef=0.01,
        verbose=0
    )
    agent.learn(total_timesteps=5000)
    agent.save(model_path)

    predicted_regimes = []
    for obs in X_predict:
        obs_tensor = agent.policy.obs_to_tensor(obs.reshape(1, -1))[0]
        distribution = agent.policy.get_distribution(obs_tensor)
        action_probs = distribution.distribution.probs.detach().cpu().numpy()[0]

        predicted_regimes.append(np.argmax(action_probs))

    true_future_returns = predict_data['DIA'].values
    true_future_regimes = np.where(true_future_returns < negative_threshold, 0,
                             np.where(true_future_returns < positive_threshold, 1, 2))

    accuracy = np.mean(predicted_regimes == true_future_regimes)
    accuracy_log.append(accuracy)

    regime_results.append(pd.DataFrame({
        'Date': predict_data['End_Date'],
        'Predicted_Regime': predicted_regimes,
        'Actual_Regime': true_future_regimes,
        'True_DIA_Return': true_future_returns
    }))

    end_time = time.time()
    print(f"Trained from {train_data['End_Date'].iloc[0].date()} to {train_data['End_Date'].iloc[-1].date()}, "
          f"Predicted starting {predict_data['End_Date'].iloc[0].date()}, Accuracy: {accuracy:.2%}, "
          f"Elapsed time: {end_time - start_time:.4f} seconds")

final_results = pd.concat(regime_results, ignore_index=True)
final_results.to_csv("high_level_predictions_with_actual.csv", index=False)
print(f"Overall accuracy: {np.mean(accuracy_log):.2%}")


In [ ]:
train_data = data.iloc[start:start + train_window].copy()
predict_data = data.iloc[start + train_window:start + train_window + predict_window].copy()

scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[features].values)
X_predict = scaler.transform(predict_data[features].values)

# Explicit regime definition (absolute returns)
regime_features = train_data[['Return_1d']].values.flatten()
regimes_current = np.where(regime_features < negative_threshold, 0,
                    np.where(regime_features < positive_threshold, 1, 2))

# Shifted target explicitly for future prediction
regimes_next_period = np.roll(regimes_current, -predict_window)
regimes_next_period_aligned = regimes_next_period[:-predict_window]
X_train_aligned = X_train[:-predict_window]

env_high = HighLevelEnv(X_train_aligned, regimes_next_period_aligned)
model_path = os.path.join(model_dir, f"ppo_model_step_{start}.zip")

agent = PPO(
    "MlpPolicy",
    env_high,
    learning_rate=1e-4,
    gamma=0.98,
    n_steps=1024,
    ent_coef=0.01,
    verbose=0
)
agent.learn(total_timesteps=5000)




In [ ]:
predict_data

In [ ]:
predicted_regimes = []
for obs in X_predict:
    obs_tensor = agent.policy.obs_to_tensor(obs.reshape(1, -1))[0]
    distribution = agent.policy.get_distribution(obs_tensor)
    action_probs = distribution.distribution.probs.detach().cpu().numpy()[0]

    predicted_regimes.append(np.argmax(action_probs))

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces

# === Explicit Data Loading ===
stock_factors = pd.read_csv("stock_factors_28_stocks.csv", parse_dates=["Date"]).set_index("Date")
daily_returns = pd.read_csv("daily_returns.csv", parse_dates=["Date"]).set_index("Date")
high_level_preds = pd.read_csv("high_level_rolling_predictions_with_actual_sequence.csv", parse_dates=["Date"]).set_index("Date")

stock_factors.index = pd.to_datetime(stock_factors.index)
daily_returns.index = pd.to_datetime(daily_returns.index)
high_level_preds.index = pd.to_datetime(high_level_preds.index)

# Parameters
train_window = 21*6
predict_window = 5
step_size = 5
window_size = 21
model_dir = "ppo_low_level_models"
os.makedirs(model_dir, exist_ok=True)

# Explicit retraining indicator
retrain_existing_model = False   # Set False for initial training, True to continue training existing models
initial_iterations = 10000       # Initial training steps
additional_iterations = 5000   # Incremental retraining steps


portfolio_results = []

predicted_regimes_series = high_level_preds['Predicted_Regime'].reindex(stock_factors.index, method='ffill')
predicted_regimes_series = predicted_regimes_series.ffill().dropna()

# Explicitly align stock_factors and daily_returns with regimes to ensure identical dates
common_dates = stock_factors.index.intersection(predicted_regimes_series.index)
stock_factors = stock_factors.loc[common_dates]
daily_returns = daily_returns.loc[common_dates]
predicted_regimes_series = predicted_regimes_series.loc[common_dates]
dates = stock_factors.index.unique()
# Explicit PPO training function
def train_or_load_agent(env, model_path, iterations, retrain=False):
    if retrain and os.path.exists(model_path):
        print(f"✅ Loading existing model explicitly from {model_path}")
        agent = PPO.load(model_path, env=env)
    else:
        print("✅ Initializing new PPO agent explicitly.")
        agent = PPO("MlpPolicy", env, verbose=0)

    print(f"✅ Explicitly training for {iterations} timesteps.")
    agent.learn(total_timesteps=iterations)

    print(f"✅ Saving model explicitly to {model_path}")
    agent.save(model_path)
    return agent

for start in range(window_size, len(dates) - train_window - predict_window, step_size):
# for start in range(window_size, 100, step_size):    
    start_time = time.time()
    train_dates = dates[start : start + train_window]
    predict_dates = dates[start + train_window : start + train_window + predict_window]

    # Explicit training data (no leakage)
    X_train = stock_factors.loc[train_dates].values
    y_train = daily_returns.loc[train_dates].values
    train_regime = predicted_regimes_series.loc[train_dates[-1]]

    # Explicit prediction data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    env_train = LowLevelPortfolioEnv(
        features=X_train_scaled,
        returns=y_train,
        regimes=np.full(len(X_train_scaled), train_regime),
        window_size=window_size,
        predict_window=predict_window
    )
    check_env(env_train)

    # Explicit model path naming
    model_path = os.path.join(model_dir, f"low_level_ppo_step_{start}.zip")

    iterations = additional_iterations if retrain_existing_model else initial_iterations
    low_level_agent = train_or_load_agent(
        env=env_train,
        model_path=model_path,
        iterations=iterations,
        retrain=retrain_existing_model
    )

    # Explicit prediction preparation (no data leakage)
    hist_start_idx = start + train_window - window_size
    hist_end_idx = start + train_window
    X_predict_hist = stock_factors.iloc[hist_start_idx:hist_end_idx].values
    y_predict_hist = daily_returns.iloc[hist_start_idx:hist_end_idx].values
    X_predict_hist_scaled = scaler.transform(X_predict_hist)
    predict_regime = predicted_regimes_series.iloc[hist_end_idx - 1]

    env_predict = LowLevelPortfolioEnv(
        features=X_predict_hist_scaled,
        returns=y_predict_hist,
        regimes=np.full(window_size, predict_regime),
        window_size=window_size,
        predict_window=predict_window
    )

    obs, _ = env_predict.reset()

    action, _ = low_level_agent.predict(obs, deterministic=True)
    action = np.clip(action, 0, 1)
    action /= np.sum(action + 1e-8)

    # Evaluate explicit reward and metrics using predicted action
    y_predict_next_period = daily_returns.loc[predict_dates].values

    next_period_returns = y_predict_next_period @ action
    cumulative_return = np.prod(1 + next_period_returns) - 1
    cvar_threshold = np.percentile(next_period_returns, env_predict.alpha * 100)
    mean_cvar_reward = np.mean(next_period_returns) - env_predict.lambda_risk * max(cvar_threshold - np.mean(next_period_returns), 0)
    
    # Explicitly store comprehensive results in DataFrame
    result_df = pd.DataFrame({
        'Train_Start_Date': [train_dates[0].date()],
        'Train_End_Date': [train_dates[-1].date()],
        'Predict_Start_Date': [predict_dates[0].date()],
        'Predict_End_Date': [predict_dates[-1].date()],
        'Mean_CVaR_Reward': [mean_cvar_reward],
        'Cumulative_Return': [cumulative_return],
    })
    
    # Explicitly add portfolio weights
    for i, asset in enumerate(daily_returns.columns):
        result_df[f'Weight_{asset}'] = action[i]
    
    portfolio_results.append(result_df)

    print(f"✅ Trained from {train_dates[0].date()} to {train_dates[-1].date()}, "
          f"allocated portfolio from {predict_dates[0].date()} to {predict_dates[-1].date()}")
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time:.4f} seconds")
# Explicit consolidation and export
if portfolio_results:
    final_allocations = pd.concat(portfolio_results)
    final_allocations.to_csv("low_level_optimal_portfolio_allocations.csv")
    print("✅ Portfolio allocations explicitly saved to 'low_level_optimal_portfolio_allocations.csv'.")
else:
    print("⚠️ No allocations generated; verify data alignment and parameters.")


In [ ]:
####################################################################################
## low level agent to include shap related metrics in the training and prediction ##
####################################################################################
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces

# === Explicit Data Loading ===
stock_factors = pd.read_csv("stock_factors_28_stocks.csv", parse_dates=["Date"]).set_index("Date")
daily_returns = pd.read_csv("daily_returns.csv", parse_dates=["Date"]).set_index("Date")
high_level_preds = pd.read_csv("high_level_rolling_predictions_with_actual_sequence.csv", parse_dates=["Date"]).set_index("Date")

stock_factors.index = pd.to_datetime(stock_factors.index)
daily_returns.index = pd.to_datetime(daily_returns.index)
high_level_preds.index = pd.to_datetime(high_level_preds.index)

# Parameters
train_window = 21*6
predict_window = 5
step_size = 5
window_size = 21
model_dir = "ppo_low_level_models"
os.makedirs(model_dir, exist_ok=True)

# Explicit retraining indicator
retrain_existing_model = False   # Set False for initial training, True to continue training existing models
initial_iterations = 5000       # Initial training steps
additional_iterations = 5000   # Incremental retraining steps


portfolio_results = []

predicted_regimes_series = high_level_preds['Predicted_Regime'].reindex(stock_factors.index, method='ffill')
predicted_regimes_series = predicted_regimes_series.ffill().dropna()

shap_df = pd.read_csv("shap_with_DIA_indicators.csv", parse_dates=["End_Date"]).set_index("End_Date")
shap_df.index = pd.to_datetime(shap_df.index)

# Explicitly define the specific numeric columns you need
selected_shap_columns = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA', 'mean_abs_shap_Stock',
    'mean_shap_Mkt-RF', 'mean_shap_SMB', 'mean_shap_HML', 'mean_shap_RMW',
    'mean_shap_CMA', 'mean_shap_Stock', 'shap_std_Mkt-RF', 'shap_std_SMB',
    'shap_std_HML', 'shap_std_RMW', 'shap_std_CMA', 'shap_std_Stock',
    'mean_over_std_Mkt-RF', 'mean_over_std_SMB', 'mean_over_std_HML',
    'mean_over_std_RMW', 'mean_over_std_CMA', 'mean_over_std_Stock',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB',
    'mean_abs_over_std_HML', 'mean_abs_over_std_RMW',
    'mean_abs_over_std_CMA', 'mean_abs_over_std_Stock',
    'Return_1d', 'Return_5d', 'Rolling_Return_21d',
    'Volatility_5d', 'Volatility_21d', 'Drawdown',
    'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'RSI_14'
]

# Explicitly filtering the dataframe
shap_df = shap_df[selected_shap_columns]
# Explicitly align all datasets
common_dates = stock_factors.index.intersection(shap_df.index).intersection(predicted_regimes_series.index)

stock_factors = stock_factors.loc[common_dates]
daily_returns = daily_returns.loc[common_dates]
predicted_regimes_series = predicted_regimes_series.loc[common_dates]
shap_df = shap_df.loc[common_dates]

dates = stock_factors.index.unique()
# Explicit PPO training function
def train_or_load_agent(env, model_path, iterations, retrain=False):
    if retrain and os.path.exists(model_path):
        print(f"✅ Loading existing model explicitly from {model_path}")
        agent = PPO.load(model_path, env=env)
    else:
        print("✅ Initializing new PPO agent explicitly.")
        agent = PPO("MlpPolicy", env, verbose=0)

    print(f"✅ Explicitly training for {iterations} timesteps.")
    agent.learn(total_timesteps=iterations)

    print(f"✅ Saving model explicitly to {model_path}")
    agent.save(model_path)
    return agent

for start in range(window_size, len(dates) - train_window - predict_window, step_size):
# for start in range(window_size, 100, step_size):    
    start_time = time.time()
    
    train_dates = dates[start : start + train_window]
    predict_dates = dates[start + train_window : start + train_window + predict_window]

    # Explicit training data (no leakage)
    X_train = stock_factors.loc[train_dates].values
    y_train = daily_returns.loc[train_dates].values
    shap_train = shap_df.loc[train_dates].values
    train_regime = predicted_regimes_series.loc[train_dates[-1]]

    # Scale stock features explicitly (SHAP features typically don't require scaling, but can be scaled if preferred)
    scaler_stock = StandardScaler()
    X_train_scaled = scaler_stock.fit_transform(X_train)

    env_train = LowLevelPortfolioEnv(
        features=X_train_scaled,
        returns=y_train,
        shap_features=shap_train,
        regimes=np.full(len(X_train_scaled), train_regime),
        window_size=window_size,
        predict_window=predict_window
    )
    check_env(env_train)

    model_path = os.path.join(model_dir, f"low_level_ppo_step_{start}.zip")
    iterations = additional_iterations if retrain_existing_model else initial_iterations

    low_level_agent = train_or_load_agent(
        env=env_train,
        model_path=model_path,
        iterations=iterations,
        retrain=retrain_existing_model
    )

    # Explicit prediction preparation (historical window)
    hist_start_idx = start + train_window - window_size
    hist_end_idx = start + train_window

    X_predict_hist = stock_factors.iloc[hist_start_idx:hist_end_idx].values
    shap_predict_hist = shap_df.iloc[hist_start_idx:hist_end_idx].values
    y_predict_hist = daily_returns.iloc[hist_start_idx:hist_end_idx].values

    X_predict_hist_scaled = scaler_stock.transform(X_predict_hist)
    predict_regime = predicted_regimes_series.iloc[hist_end_idx - 1]

    env_predict = LowLevelPortfolioEnv(
        features=X_predict_hist_scaled,
        returns=y_predict_hist,
        shap_features=shap_predict_hist,
        regimes=np.full(window_size, predict_regime),
        window_size=window_size,
        predict_window=predict_window
    )

    obs, _ = env_predict.reset()
    action, _ = low_level_agent.predict(obs, deterministic=True)
    action = np.clip(action, 0, 1)
    action /= np.sum(action + 1e-8)

    # Evaluate reward explicitly
    y_predict_next_period = daily_returns.loc[predict_dates].values
    next_period_returns = y_predict_next_period @ action
    cumulative_return = np.prod(1 + next_period_returns) - 1

    result_df = pd.DataFrame({
        'Train_Start_Date': [train_dates[0].date()],
        'Train_End_Date': [train_dates[-1].date()],
        'Predict_Start_Date': [predict_dates[0].date()],
        'Predict_End_Date': [predict_dates[-1].date()],
        'Cumulative_Return': [cumulative_return],
    })

    for i, asset in enumerate(daily_returns.columns):
        result_df[f'Weight_{asset}'] = action[i]

    portfolio_results.append(result_df)

    print(f"✅ Trained from {train_dates[0].date()} to {train_dates[-1].date()}, "
          f"allocated portfolio from {predict_dates[0].date()} to {predict_dates[-1].date()}")
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time:.4f} seconds")
# Explicit consolidation and export
if portfolio_results:
    final_allocations = pd.concat(portfolio_results)
    final_allocations.to_csv("low_level_optimal_portfolio_allocations.csv")
    print("✅ Portfolio allocations explicitly saved to 'low_level_optimal_portfolio_allocations.csv'.")
else:
    print("⚠️ No allocations generated; verify data alignment and parameters.")


In [ ]:
shap_df

In [ ]:
len(dates) - train_window - predict_window

In [ ]:
# combined code use this!!
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import time

# Explicit Data Loading
shap_df = pd.read_csv("shap_with_DIA_indicators.csv", parse_dates=["End_Date"])
dia_returns_df = pd.read_csv("daily_returns_DIA.csv", parse_dates=["Date"]).set_index("Date")
stock_returns_df = pd.read_csv("daily_returns.csv", parse_dates=["Date"]).set_index("Date")
stock_factors = pd.read_csv("stock_factors_28_stocks.csv", parse_dates=["Date"]).set_index("Date")

data = pd.merge(shap_df, dia_returns_df[['DIA']], left_on="End_Date", right_index=True).dropna()

# features = [col for col in shap_df.columns if col not in ["End_Date", "DIA"]]
features = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA', 'mean_abs_shap_Stock',
    'mean_shap_Mkt-RF', 'mean_shap_SMB', 'mean_shap_HML', 'mean_shap_RMW',
    'mean_shap_CMA', 'mean_shap_Stock', 'shap_std_Mkt-RF', 'shap_std_SMB',
    'shap_std_HML', 'shap_std_RMW', 'shap_std_CMA', 'shap_std_Stock',
    'mean_over_std_Mkt-RF', 'mean_over_std_SMB', 'mean_over_std_HML',
    'mean_over_std_RMW', 'mean_over_std_CMA', 'mean_over_std_Stock',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB',
    'mean_abs_over_std_HML', 'mean_abs_over_std_RMW',
    'mean_abs_over_std_CMA', 'mean_abs_over_std_Stock',
    'Return_1d', 'Return_5d', 'Rolling_Return_21d',
    'Volatility_5d', 'Volatility_21d', 'Drawdown',
    'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'RSI_14'
]
# Parameters
train_window, predict_window, step_size, consistency_window, window_size = 360, 5, 5, 5, 21
portfolio_results, accuracy_log, regime_results = [], [], []

# for start in range(0, len(data) - train_window - predict_window - window_size, step_size):
for start in range(0, 60, step_size):

    start_time = time.time()

    # Data slicing (no leakage)
    train_data = data.iloc[start:start + train_window].copy()
    predict_data = data.iloc[start + train_window:start + train_window + predict_window].copy()

    X_train_hl = train_data[features].values
    X_predict_hl = predict_data[features].values

    scaler_hl = StandardScaler()
    X_train_hl_scaled = scaler_hl.fit_transform(X_train_hl)
    X_predict_hl_scaled = scaler_hl.transform(X_predict_hl)

    # Regime modeling explicitly on training data
    gmm = GaussianMixture(n_components=3, random_state=42)
    regimes_train = gmm.fit_predict(X_train_hl_scaled)

    regimes_train_next = np.roll(regimes_train, -step_size)
    X_train_aligned = X_train_hl_scaled[:-step_size]
    regimes_train_aligned = regimes_train_next[:-step_size]

    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train_aligned, regimes_train_aligned)
    supervised_pred = clf.predict(X_predict_hl_scaled)

    env_high = HighLevelEnv(X_train_hl_scaled, regimes_train_next, consistency_window)
    high_level_agent = PPO("MlpPolicy", env_high, verbose=0)
    high_level_agent.learn(total_timesteps=500)

    predicted_regimes = []
    for obs in X_predict_hl_scaled:
        pred_regime, _ = high_level_agent.predict(obs, deterministic=True)
        predicted_regimes.append(int(pred_regime))

    final_predicted_regimes = [
        ppo_pred if ppo_pred == sup_pred else sup_pred
        for ppo_pred, sup_pred in zip(predicted_regimes, supervised_pred)
    ]

    # Accuracy logging
    regimes_predict_actual = gmm.predict(X_predict_hl_scaled)
    accuracy = np.mean(final_predicted_regimes == regimes_predict_actual)
    accuracy_log.append(accuracy)

    # Store predicted and actual regimes explicitly
    regime_results.append(pd.DataFrame({
        'Date': predict_data['End_Date'].values,
        'Predicted_Regime': final_predicted_regimes,
        'Actual_Regime': regimes_predict_actual
    }))

    # Low-level Data Alignment
    train_dates = train_data['End_Date']
    predict_dates = predict_data['End_Date']

    X_train_ll = stock_factors.loc[train_dates].values
    y_train_ll = stock_returns_df.loc[train_dates].values

    X_predict_ll = stock_factors.loc[predict_dates].values
    y_predict_ll = stock_returns_df.loc[predict_dates].values

    scaler_ll = StandardScaler()
    X_train_ll_scaled = scaler_ll.fit_transform(X_train_ll)
    X_predict_ll_scaled = scaler_ll.transform(X_predict_ll)

    train_regime = final_predicted_regimes[0]
    predict_regime = final_predicted_regimes[-1]
    print(f'train_regime = {train_regime} and predict_regime = {predict_regime}')
    # env_low_train = LowLevelPortfolioEnv(
    #     X_train_ll_scaled, y_train_ll,
    #     regimes=np.full(len(X_train_ll_scaled), train_regime),
    #     predict_window=predict_window, window_size=window_size
    # )
    # check_env(env_low_train)
    # 
    # low_level_agent = PPO("MlpPolicy", env_low_train, verbose=0)
    # low_level_agent.learn(total_timesteps=500)
    # 
    # predict_start_idx = stock_factors.index.get_loc(predict_dates.iloc[0])
    # 
    # X_predict_hist = stock_factors.iloc[
    #     predict_start_idx - window_size : predict_start_idx
    # ].values
    # 
    # y_predict_hist = stock_returns_df.iloc[
    #     predict_start_idx - window_size : predict_start_idx
    # ].values
    # 
    # X_predict_hist_scaled = scaler_ll.transform(X_predict_hist)
    # 
    # env_low_predict = LowLevelPortfolioEnv(
    #     X_predict_hist_scaled, y_predict_hist,
    #     regimes=np.full(len(X_predict_hist_scaled), predict_regime),
    #     predict_window=predict_window, window_size=window_size
    # )
    # 
    # obs, _ = env_low_predict.reset()
    # action, _ = low_level_agent.predict(obs, deterministic=True)
    # action = np.clip(action, 0, 1)
    # action /= np.sum(action + 1e-8)
    # 
    # allocation_df = pd.DataFrame(
    #     [action] * len(predict_dates),
    #     columns=stock_returns_df.columns,
    #     index=predict_dates
    # )
    # portfolio_results.append(allocation_df)

    end_time = time.time()

    # Logging
    print(f"Step {start}: High-level accuracy: {accuracy:.2%}, Time elapsed: {end_time - start_time:.2f}s")

# Final consolidation
# final_allocations = pd.concat(portfolio_results)
# final_allocations.to_csv("combined_portfolio_allocations.csv")
# 
# accuracy_overall = np.mean(accuracy_log)
# print(f"Overall high-level prediction accuracy: {accuracy_overall:.2%}")
# 
# final_regime_results = pd.concat(regime_results)
# final_regime_results.to_csv("regime_predictions_actuals.csv", index=False)





In [ ]:
final_predicted_regimes

In [ ]:
HighLevelEnv

In [ ]:
# Correctly shift regimes explicitly to predict future regime
regimes_train_next = np.roll(regimes_train, -step_size)

# Explicitly drop the last 'step_size' samples with unknown future regimes
X_train_aligned = X_train[:-step_size]  # Explicitly drop last rows to align
regimes_train_aligned = regimes_train_next[:-step_size]

# Explicit supervised baseline without look-ahead bias
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train_aligned, regimes_train_aligned)

# Explicitly predict with your fitted classifier
supervised_pred = clf.predict(X_predict)
supervised_pred

In [ ]:

hmm.fit(X_train)


regimes_train = hmm.predict(X_train)
regimes_predict_actual = hmm.predict(X_predict)


# Correctly shift regimes explicitly to predict future regime
regimes_train_next = np.roll(regimes_train, -step_size)

# Explicitly drop the last 'step_size' samples with unknown future regimes
X_train_aligned = X_train[:-step_size]  # Explicitly drop last rows to align
regimes_train_aligned = regimes_train_next[:-step_size]

In [ ]:
import hmmlearn
print("hmmlearn updated version:", hmmlearn.__version__)

In [ ]:
# Explicit Rolling-Window Training & Prediction
portfolio_results = []

# Ensure explicitly proper datetime indexing
data.index = pd.to_datetime(data.index)
dates = data.index.unique()

train_window = 252
predict_window = 5
step_size = 5

for start in range(0, len(dates) - train_window - predict_window, step_size):
    train_dates = dates[start : start + train_window]
    predict_dates = dates[start + train_window : start + train_window + predict_window]

    # Check explicitly if data slices are empty
    if len(train_dates) < train_window or len(predict_dates) < predict_window:
        print(f"Insufficient data at start index {start}, skipping explicitly.")
        continue

    # Explicitly slice data
    train_data = data.loc[train_dates].dropna()
    predict_data = data.loc[predict_dates].dropna()

    if train_data.empty or predict_data.empty:
        print(f"No available data in train or predict window at index {start}, explicitly skipping.")
        continue

    # Split explicitly into features, returns, and regimes
    scaler = StandardScaler()
    X_train_features = scaler.fit_transform(train_data.filter(like="_factors"))
    y_train_returns = train_data.filter(like="_returns").values
    train_regimes = train_data["Predicted_Regime"].values

    X_predict_features = scaler.transform(predict_data.filter(like="_factors"))
    y_predict_returns = predict_data.filter(like="_returns").values
    predict_regimes = predict_data["Predicted_Regime"].values

    if len(train_regimes) <= 5 or len(y_train_returns) <= 5:
        print(f"Not enough data points after filtering at index {start}, explicitly skipping.")
        continue

    # Create and train low-level agent explicitly
    env = LowLevelPortfolioEnv(X_train_features, y_train_returns, train_regimes)
    check_env(env)

    low_level_agent = PPO("MlpPolicy", env, verbose=0)
    low_level_agent.learn(total_timesteps=50000)

    # Explicit out-of-sample portfolio allocation
    env_predict = LowLevelPortfolioEnv(X_predict_features, y_predict_returns, predict_regimes)
    obs, _ = env_predict.reset()

    allocations = []
    for day in range(len(predict_dates)):
        action, _ = low_level_agent.predict(obs, deterministic=True)
        action /= np.sum(action + 1e-8)
        allocations.append(action)

        obs, reward, terminated, truncated, info = env_predict.step(action)
        if terminated:
            break

    if allocations:
        allocations_df = pd.DataFrame(
            allocations,
            columns=predict_data.filter(like="_returns").columns,
            index=predict_dates[:len(allocations)]
        )

        portfolio_results.append(allocations_df)

        print(f"✅ Trained explicitly from {train_dates[0].date()} to {train_dates[-1].date()}, "
              f"allocated portfolio starting {predict_dates[0].date()}")

# Final explicit consolidation with explicit check
if portfolio_results:
    final_allocations = pd.concat(portfolio_results)
    final_allocations.to_csv("low_level_portfolio_allocations.csv")
    print("✅ Explicit rolling-window training and allocations completed.")
else:
    print("⚠️ No allocations generated explicitly; check data alignment and window settings.")


In [ ]:
# start from here
# +-------------------------------+           +-------------------------------+
# | DIA SHAP + Technical Indicators|           |    Daily Returns (28 Stocks)  |
# | (from shap_with_DIA_indicators.csv)        |  (from daily_returns.csv)     |
# +--------------+----------------+           +---------------+---------------+
#                |                                            |
#                v                                            v
#         +---------------------+                  +--------------------------+
#         |  High-Level Agent   |----------------->|   Low-Level PPO Agents   |
#         |  (PPO classifier)   | Regime Signal    | (trained per market regime)|
#         +---------------------+                  +--------------------------+
#                |                                            |
#                v                                            v
#         Regime prediction                         Optimal portfolio weights
#                                                   (Mean-CVaR reward explicitly benchmarked vs naive)


import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM  # explicitly re-import after restart
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces
import time
import logging
# Load Data Explicitly
shap_df = pd.read_csv("shap_with_DIA_indicators.csv", parse_dates=["End_Date"])
returns_df = pd.read_csv("daily_returns.csv", parse_dates=["Date"]).set_index("Date")
stock_factors = pd.read_csv("stock_factors_28_stocks.csv", parse_dates=["Date"]).set_index("Date")

data = pd.merge(shap_df, returns_df, left_on="End_Date", right_index=True).dropna()
data.index = pd.to_datetime(data['End_Date'])

# Explicit alignment of all data by dates
combined_data = data.join(stock_factors, how='inner').dropna()

features = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA', 'mean_abs_shap_Stock',
    'mean_shap_Mkt-RF', 'mean_shap_SMB', 'mean_shap_HML', 'mean_shap_RMW',
    'mean_shap_CMA', 'mean_shap_Stock', 'shap_std_Mkt-RF', 'shap_std_SMB',
    'shap_std_HML', 'shap_std_RMW', 'shap_std_CMA', 'shap_std_Stock',
    'mean_over_std_Mkt-RF', 'mean_over_std_SMB', 'mean_over_std_HML',
    'mean_over_std_RMW', 'mean_over_std_CMA', 'mean_over_std_Stock',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB',
    'mean_abs_over_std_HML', 'mean_abs_over_std_RMW',
    'mean_abs_over_std_CMA', 'mean_abs_over_std_Stock',
    'Return_1d', 'Return_5d', 'Rolling_Return_21d',
    'Volatility_5d', 'Volatility_21d', 'Drawdown',
    'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'RSI_14'
]

train_window = 252
predict_window = 21
step_size = 21

portfolio_results = []
high_level_results = []

# for start in range(0, len(combined_data) - train_window - predict_window, step_size):
for start in range(0, 200, step_size):

    start_time = time.time()

    # === Explicit Data slicing ===
    train_data = combined_data.iloc[start : start + train_window].copy()
    predict_data = combined_data.iloc[start + train_window : start + train_window + predict_window].copy()

    # High-Level HMM explicitly fitted on historical data
    # === Step 2: Explicit HMM regime labeling ONLY on past training data ===
    scaler = StandardScaler()
    X_train = scaler.fit_transform(train_data[features])
    X_predict = scaler.transform(predict_data[features])
    
    X_train = np.nan_to_num(X_train)
    X_predict = np.nan_to_num(X_predict)
    
    hmm = GaussianHMM(
        n_components=3,         # explicitly fewer states, more stable
        covariance_type="diag", # explicitly more stable covariance
        n_iter=5000,            # explicit sufficient iterations for convergence
        tol=1e-4,               # explicit convergence tolerance
        random_state=42
    )

    hmm.fit(X_train)
    
    
    regimes_train = hmm.predict(X_train)
    regimes_predict_actual = hmm.predict(X_predict)
    
    
    # Correctly shift regimes explicitly to predict future regime
    regimes_train_next = np.roll(regimes_train, -step_size)
    
    # Explicitly drop the last 'step_size' samples with unknown future regimes
    X_train_aligned = X_train[:-step_size]  # Explicitly drop last rows to align
    regimes_train_aligned = regimes_train_next[:-step_size]
    
    # Explicit supervised baseline without look-ahead bias
    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train_aligned, regimes_train_aligned)
    
    # Explicitly predict with your fitted classifier
    supervised_pred = clf.predict(X_predict)
    # PPO Environment explicitly trained to predict the regime 5-days ahead

    env_high = HighLevelEnv(X_train, regimes_train_next, train_returns)

    high_level_agent = PPO(
        "MlpPolicy",
        env_high,
        learning_rate=1e-4,
        gamma=0.95,
        ent_coef=0.01,
        n_steps=512,
        verbose=0
    )
    high_level_agent.learn(total_timesteps=100000)
    
    # Explicit predictions
    predicted_regimes = []
    for obs in X_predict.astype(np.float32):
        pred_regime, _ = high_level_agent.predict(obs, deterministic=True)
        predicted_regimes.append(int(pred_regime))
    
    # Ensemble explicitly with supervised baseline for robustness
    final_predicted_regimes = [
        ppo_pred if ppo_pred == sup_pred else sup_pred
        for ppo_pred, sup_pred in zip(predicted_regimes, supervised_pred)
    ]

    predict_dates = predict_data['End_Date'].values

    
    high_level_results.append(pd.DataFrame({
        'Date': predict_dates,
        'Predicted_Regime': predicted_regimes,
        'Actual_Regime': regimes_predict_actual
    }))

    # === Low-Level Agent explicitly uses predicted regimes ===
    factor_cols = stock_factors.columns
    return_cols = returns_df.columns

    train_stock_features = scaler.fit_transform(train_data[factor_cols])
    train_returns = train_data[return_cols].values
    train_predicted_regimes = regimes_train_next  # predicted regimes from high-level

    predict_stock_features = scaler.transform(predict_data[factor_cols])
    predict_returns = predict_data[return_cols].values
    predict_regimes = predicted_regimes  # explicitly predicted from high-level model

    # Low-Level PPO Environment explicitly
    class LowLevelEnv(gym.Env):
        def __init__(self, features, returns, regimes, window_size=21):
            super().__init__()
            min_len = min(len(features), len(returns), len(regimes))
            self.features = features[:min_len].astype(np.float32)
            self.returns = returns[:min_len].astype(np.float32)
            self.regimes = regimes[:min_len]
    
            self.window_size = window_size
            self.n_assets = returns.shape[1]
            self.index = self.window_size
    
            obs_shape = (window_size * self.features.shape[1]) + 3
            self.observation_space = spaces.Box(-np.inf, np.inf, shape=(obs_shape,), dtype=np.float32)
            self.action_space = spaces.Box(0, 1, shape=(self.n_assets,), dtype=np.float32)
            self.regime_encoding = {0: [1,0,0], 1: [0,1,0], 2: [0,0,1]}
    
            logging.info(f"Initialized LowLevelEnv: features length {len(self.features)}, returns length {len(self.returns)}, regimes length {len(self.regimes)}.")
    
        def reset(self, seed=None, options=None):
            super().reset(seed=seed)
            self.index = self.window_size
            if self.index >= len(self.features):
                logging.error(f"Reset failed: index {self.index} >= features length {len(self.features)}")
                raise IndexError("Reset index out of bounds.")
            return self._get_obs(), {}
    
        def _get_obs(self):
            recent_features = self.features[self.index - self.window_size:self.index].flatten()
            regime_idx = min(self.index, len(self.regimes)-1)  # Ensure in bounds explicitly
            regime_onehot = self.regime_encoding[self.regimes[regime_idx]]
            return np.concatenate([recent_features, regime_onehot]).astype(np.float32)
    
        def step(self, action):
            action = np.clip(action, 0, 1)
            action /= action.sum() + 1e-8
            ret = np.dot(action, self.returns[self.index])
            reward = ret
            self.index += 1
            terminated = self.index >= len(self.returns)-1
            obs = self._get_obs() if not terminated else np.zeros(self.observation_space.shape, dtype=np.float32)
            return obs, reward, terminated, False, {}


# Add these checks explicitly before initializing LowLevelEnv
    if len(train_predicted_regimes) <= 21 or len(train_stock_features) <= 21:
        logging.warning(f"Skipping low-level training at iteration {start} due to insufficient training data.")
        continue
    
    if len(predict_regimes) <= 21 or len(predict_stock_features) <= 21:
        logging.warning(f"Skipping low-level prediction at iteration {start} due to insufficient prediction data.")
        continue
    
    env_low = LowLevelEnv(train_stock_features, train_returns, train_predicted_regimes)
    low_level_agent = PPO("MlpPolicy", env_low, verbose=0)
    low_level_agent.learn(total_timesteps=50000)
    
    env_low_pred = LowLevelEnv(predict_stock_features, predict_returns, predict_regimes)
    obs, _ = env_low_pred.reset()
    
    allocations = []
    for _ in range(len(predict_dates)):
        action, _ = low_level_agent.predict(obs, deterministic=True)
        action /= np.sum(action + 1e-8)
        allocations.append(action)
        obs, reward, terminated, truncated, _ = env_low_pred.step(action)
        if terminated:
            break


    alloc_df = pd.DataFrame(allocations, columns=return_cols, index=predict_dates[:len(allocations)])
    portfolio_results.append(alloc_df)
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time:.4f} seconds")
    
# Final Results Saving explicitly
final_allocations = pd.concat(portfolio_results)
final_allocations.to_csv("integrated_portfolio_allocations.csv")
final_high_level = pd.concat(high_level_results)
final_high_level.to_csv("integrated_high_level_predictions.csv")

print("✅ Fully explicit integrated rolling-window training complete.")


In [ ]:
high_level_results

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import gym
from gym import spaces
import torch
import os

# === Data Load ===
shap_file = "shap_with_DIA_indicators.csv"
returns_file = "daily_returns.csv"

shap_df = pd.read_csv(shap_file, parse_dates=["End_Date"])
returns_df = pd.read_csv(returns_file, parse_dates=["Date"])
returns_df.set_index("Date", inplace=True)

data = pd.merge(shap_df, returns_df, how="inner", left_on="End_Date", right_index=True)

# === Market Regime Detection using HMM ===
features = [
    "mean_abs_shap_Mkt-RF", "mean_abs_shap_SMB", "mean_abs_shap_HML", 
    "mean_abs_shap_RMW", "mean_abs_shap_CMA", "mean_abs_shap_Stock",
    "Return_1d", "Volatility_5d", "RSI_14", "MACD", "ATR_14"
]

X = data[features].fillna(method="ffill").dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

hmm = GaussianHMM(n_components=3, covariance_type="full", n_iter=2000, random_state=42)
regime_labels = hmm.fit_predict(X_scaled)

data = data.iloc[-len(regime_labels):].copy()
data["Regime"] = regime_labels

# === PPO Environments ===
class HighLevelEnv(gym.Env):
    def __init__(self, X, regimes):
        super().__init__()
        self.X = X
        self.regimes = regimes
        self.index = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X.shape[1],))
        self.action_space = spaces.Discrete(3)

    def reset(self):
        self.index = 0
        return self.X[self.index]

    def step(self, action):
        reward = 1 if action == self.regimes[self.index] else -1
        self.index += 1
        done = self.index >= len(self.X)-1
        obs = self.X[self.index] if not done else np.zeros(self.X.shape[1])
        return obs, reward, done, {}

class LowLevelEnv(gym.Env):
    def __init__(self, returns):
        super().__init__()
        self.returns = returns
        self.n_assets = returns.shape[1]
        self.index = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets,))
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,))

    def reset(self):
        self.index = 0
        return self.returns[self.index]

    def step(self, action):
        action = np.clip(action, 0, 1)
        action /= np.sum(action + 1e-8)
        portfolio_return = np.dot(action, self.returns[self.index])
        cvar_threshold = np.percentile(self.returns[self.index], 5)
        reward = portfolio_return - 0.5 * max(cvar_threshold - portfolio_return, 0)
        self.index += 1
        done = self.index >= len(self.returns)-1
        obs = self.returns[self.index] if not done else np.zeros(self.n_assets)
        return obs, reward, done, {}

# === Train High-Level PPO ===
high_level_env = HighLevelEnv(X_scaled, regime_labels)
check_env(high_level_env)

high_level_model = PPO("MlpPolicy", high_level_env, verbose=1)
high_level_model.learn(total_timesteps=20000)  # Recommended steps for robust regime prediction
high_level_model.save("high_level_ppo_agent")

# === Train Low-Level PPO (one per regime) ===
low_level_agents = {}
returns_only = data.iloc[:, -29:-1]  # Ensure exactly 28 stocks, excluding 'Regime' column
returns_scaled = StandardScaler().fit_transform(returns_only)

for regime in range(3):
    regime_data = returns_scaled[data["Regime"] == regime]
    if len(regime_data) < 10:
        print(f"Regime {regime} has insufficient data; skipping.")
        continue

    env = LowLevelEnv(regime_data)
    check_env(env)
    model = PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=20000)  # Recommended for financial stability
    model.save(f"low_level_ppo_agent_regime_{regime}")
    low_level_agents[regime] = model
    print(f"✅ Saved PPO for regime {regime}")

# === Evaluation Logging ===
evaluation_results = []

for i in range(len(X_scaled)):
    obs = X_scaled[i]
    predicted_regime, _ = high_level_model.predict(obs)
    if predicted_regime in low_level_agents:
        agent = low_level_agents[predicted_regime]
        stock_obs = returns_scaled[i].reshape(1, -1)
        allocation, _ = agent.predict(stock_obs)
        allocation = np.clip(allocation, 0, 1)
        allocation /= np.sum(allocation + 1e-8)
        daily_return = np.dot(allocation, returns_scaled[i])
        evaluation_results.append({
            "Date": data["End_Date"].iloc[i],
            "Predicted_Regime": predicted_regime,
            "True_Regime": regime_labels[i],
            "Daily_Return": daily_return
        })

evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("hierarchical_rl_evaluation.csv", index=False)

print("✅ HRL Pipeline complete. Models and evaluation saved.")


In [ ]:
class HighLevelEnv(gym.Env):
    def __init__(self, X, regimes, returns_scaled, low_level_agents, alpha=0.5, lambda_risk=0.5):
        super().__init__()
        self.X = X
        self.regimes = regimes
        self.returns_scaled = returns_scaled
        self.low_level_agents = low_level_agents
        self.alpha = alpha
        self.lambda_risk = lambda_risk
        self.index = 0
        self.n_features = X.shape[1]

        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_features,))
        self.action_space = spaces.Discrete(3)

    def reset(self):
        self.index = 0
        return self.X[self.index]

    def step(self, action):
        true_regime = self.regimes[self.index]

        # Classification accuracy reward component
        classification_reward = 1 if action == true_regime else -1

        # Financial reward component from low-level agent
        if action in self.low_level_agents:
            low_level_agent = self.low_level_agents[action]
            stock_obs = self.returns_scaled[self.index].reshape(1, -1)
            allocation, _ = low_level_agent.predict(stock_obs, deterministic=True)
            allocation = np.clip(allocation, 0, 1)
            allocation /= np.sum(allocation + 1e-8)

            daily_return = np.dot(allocation, self.returns_scaled[self.index])
            cvar_threshold = np.percentile(self.returns_scaled[self.index], 5)
            cvar_penalty = max(cvar_threshold - daily_return, 0)

            financial_reward = daily_return - self.lambda_risk * cvar_penalty
        else:
            financial_reward = -1  # Penalize heavily if no matching low-level agent

        # Combine rewards
        reward = self.alpha * classification_reward + (1 - self.alpha) * financial_reward

        # Proceed to next step
        self.index += 1
        done = self.index >= len(self.X) - 1
        obs = self.X[self.index] if not done else np.zeros(self.n_features)

        return obs, reward, done, {}


In [ ]:
# === Train Low-Level PPO Agents ===
low_level_agents = {}
returns_only = data.iloc[:, -29:-1]  # exactly 28 stocks
returns_scaled = StandardScaler().fit_transform(returns_only)

for regime in range(3):
    regime_data = returns_scaled[data["Regime"] == regime]
    if len(regime_data) < 10:
        print(f"Regime {regime} insufficient data; skipping.")
        continue

    env = LowLevelEnv(regime_data)
    check_env(env)
    model = PPO("MlpPolicy", env, verbose=0)
    model.learn(total_timesteps=20000)
    model.save(f"low_level_ppo_agent_regime_{regime}")
    low_level_agents[regime] = model
    print(f"✅ Saved low-level PPO agent for regime {regime}")

# === Train Enhanced High-Level PPO ===
alpha = 0.5  # Balanced accuracy vs financial return
lambda_risk = 0.5  # Risk aversion hyperparameter

high_level_env = HighLevelEnv(X_scaled, regime_labels, returns_scaled, low_level_agents, alpha, lambda_risk)
check_env(high_level_env)

high_level_model = PPO("MlpPolicy", high_level_env, verbose=1)
high_level_model.learn(total_timesteps=20000)
high_level_model.save("enhanced_high_level_ppo_agent")
print("✅ Enhanced high-level PPO agent trained and saved.")


In [ ]:
# === Enhanced Evaluation Logging ===
evaluation_results = []

for i in range(len(X_scaled)):
    obs = X_scaled[i]
    predicted_regime, _ = high_level_model.predict(obs, deterministic=True)
    true_regime = regime_labels[i]

    # Get financial reward from selected low-level agent
    if predicted_regime in low_level_agents:
        low_level_agent = low_level_agents[predicted_regime]
        stock_obs = returns_scaled[i].reshape(1, -1)
        allocation, _ = low_level_agent.predict(stock_obs, deterministic=True)
        allocation = np.clip(allocation, 0, 1)
        allocation /= np.sum(allocation + 1e-8)
        daily_return = np.dot(allocation, returns_scaled[i])

        # Risk-adjusted measure
        cvar_threshold = np.percentile(returns_scaled[i], 5)
        cvar_penalty = max(cvar_threshold - daily_return, 0)
        financial_reward = daily_return - lambda_risk * cvar_penalty
    else:
        daily_return = financial_reward = None

    evaluation_results.append({
        "Date": data["End_Date"].iloc[i],
        "Predicted_Regime": predicted_regime,
        "True_Regime": true_regime,
        "Classification_Reward": 1 if predicted_regime == true_regime else -1,
        "Daily_Return": daily_return,
        "Financial_Reward": financial_reward,
        "Combined_High_Level_Reward": alpha * (1 if predicted_regime == true_regime else -1) + (1 - alpha) * (financial_reward if financial_reward is not None else -1)
    })

evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("enhanced_hierarchical_rl_evaluation.csv", index=False)
print("✅ Enhanced evaluation logged and saved.")


In [ ]:
# full code implement
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
import gym
from gym import spaces
import torch
import os

# === Data Load ===
shap_file = "shap_with_DIA_indicators.csv"
returns_file = "daily_returns.csv"

shap_df = pd.read_csv(shap_file, parse_dates=["End_Date"])
returns_df = pd.read_csv(returns_file, parse_dates=["Date"])
returns_df.set_index("Date", inplace=True)

data = pd.merge(shap_df, returns_df, how="inner", left_on="End_Date", right_index=True)

# === Market Regime Detection using HMM ===
features = [
    "mean_abs_shap_Mkt-RF", "mean_abs_shap_SMB", "mean_abs_shap_HML",
    "mean_abs_shap_RMW", "mean_abs_shap_CMA", "mean_abs_shap_Stock",
    "Return_1d", "Volatility_5d", "RSI_14", "MACD", "ATR_14"
]

X = data[features].fillna(method="ffill").dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

hmm = GaussianHMM(n_components=3, covariance_type="full", n_iter=2000, random_state=42)
regime_labels = hmm.fit_predict(X_scaled)

data = data.iloc[-len(regime_labels):].copy()
data["Regime"] = regime_labels

# === PPO Environments ===
class HighLevelEnv(gym.Env):
    def __init__(self, X, regimes):
        super().__init__()
        self.X = X
        self.regimes = regimes
        self.index = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X.shape[1],))
        self.action_space = spaces.Discrete(3)

    def reset(self):
        self.index = 0
        return self.X[self.index]

    def step(self, action):
        reward = 1 if action == self.regimes[self.index] else -1
        self.index += 1
        done = self.index >= len(self.X)-1
        obs = self.X[self.index] if not done else np.zeros(self.X.shape[1])
        return obs, reward, done, {}

class LowLevelBenchmarkEnv(gym.Env):
    def __init__(self, returns, window_size=21, lambda_risk=0.5, alpha=0.05):
        super().__init__()
        self.returns = returns
        self.window_size = window_size
        self.lambda_risk = lambda_risk
        self.alpha = alpha
        self.n_assets = returns.shape[1]
        self.index = window_size
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(window_size, self.n_assets))
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,))

    def reset(self):
        self.index = self.window_size
        return self.returns[self.index-self.window_size:self.index]

    def step(self, action):
        action = np.clip(action, 0, 1)
        action /= np.sum(action + 1e-8)

        # Agent and naive returns
        agent_return = np.dot(action, self.returns[self.index])
        naive_weights = np.array([1/self.n_assets] * self.n_assets)
        naive_return = np.dot(naive_weights, self.returns[self.index])

        # Excess return explicitly benchmarking naive portfolio
        excess_return = agent_return - naive_return

        historical_returns = self.returns[self.index-self.window_size:self.index] @ action
        cvar_threshold = np.percentile(historical_returns, 100*self.alpha)
        cvar_penalty = max(cvar_threshold - agent_return, 0)

        # Reward explicitly includes naive portfolio benchmark
        reward = excess_return - self.lambda_risk * cvar_penalty

        self.index += 1
        done = self.index >= len(self.returns)-1
        obs = self.returns[self.index-self.window_size:self.index] if not done else np.zeros((self.window_size, self.n_assets))

        return obs, reward, done, {
            "agent_return": agent_return,
            "naive_return": naive_return,
            "excess_return": excess_return,
            "cvar_penalty": cvar_penalty
        }

# === Train High-Level PPO ===
high_level_env = HighLevelEnv(X_scaled, regime_labels)
check_env(high_level_env)
high_level_model = PPO("MlpPolicy", high_level_env, verbose=0)
high_level_model.learn(total_timesteps=20000)
high_level_model.save("high_level_ppo_agent")

# === Train Low-Level PPO (one per regime) ===
low_level_agents = {}
returns_only = data.iloc[:, -29:-1]  # exactly 28 stocks
returns_scaled = StandardScaler().fit_transform(returns_only)

for regime in range(3):
    regime_data = returns_scaled[data["Regime"] == regime]
    if len(regime_data) < 50:
        print(f"Regime {regime} has insufficient data; skipping.")
        continue

    env = LowLevelBenchmarkEnv(regime_data)
    check_env(env)
    eval_callback = EvalCallback(env, best_model_save_path=f"./best_low_level_agent_{regime}",
                                 eval_freq=5000, deterministic=True, render=False)

    model = PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=50000, callback=eval_callback)
    model.save(f"low_level_ppo_agent_regime_{regime}")
    low_level_agents[regime] = model
    print(f"✅ Saved PPO for regime {regime}")

# === Evaluation Logging (Explicit Benchmark) ===
evaluation_results = []

for i in range(len(X_scaled)):
    obs = X_scaled[i]
    predicted_regime, _ = high_level_model.predict(obs, deterministic=True)
    true_regime = regime_labels[i]

    if predicted_regime in low_level_agents:
        low_level_agent = low_level_agents[predicted_regime]
        stock_obs = returns_scaled[i:i+21]
        if len(stock_obs) < 21:
            continue

        action, _ = low_level_agent.predict(stock_obs, deterministic=True)
        action /= np.sum(action + 1e-8)
        agent_return = np.dot(action, returns_scaled[i])

        naive_weights = np.array([1/28]*28)
        naive_return = np.dot(naive_weights, returns_scaled[i])

        evaluation_results.append({
            "Date": data["End_Date"].iloc[i],
            "Predicted_Regime": predicted_regime,
            "True_Regime": true_regime,
            "Agent_Return": agent_return,
            "Naive_Return": naive_return,
            "Excess_Return": agent_return - naive_return
        })

evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("hierarchical_rl_evaluation_with_benchmark.csv", index=False)

print("✅ Training and evaluation complete. Results saved explicitly.")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
import gym
from gym import spaces
import os

# === Step 1: Data Preprocessing ===
shap_df = pd.read_csv("shap_with_DIA_indicators.csv", parse_dates=["End_Date"])
returns_df = pd.read_csv("daily_returns.csv", parse_dates=["Date"])
returns_df.set_index("Date", inplace=True)

data = pd.merge(shap_df, returns_df, how="inner", left_on="End_Date", right_index=True)

# === Step 2: Market Regime Identification (HMM) ===
features = [
    "mean_abs_shap_Mkt-RF", "mean_abs_shap_SMB", "mean_abs_shap_HML",
    "mean_abs_shap_RMW", "mean_abs_shap_CMA", "mean_abs_shap_Stock",
    "Return_1d", "Volatility_5d", "RSI_14", "MACD", "ATR_14"
]

X = data[features].fillna(method="ffill").dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

hmm = GaussianHMM(n_components=3, covariance_type="full", n_iter=2000, random_state=42)
regime_labels = hmm.fit_predict(X_scaled)

data = data.iloc[-len(regime_labels):].copy()
data["Regime"] = regime_labels

# === Step 3: High-Level PPO Agent ===
class HighLevelEnv(gym.Env):
    def __init__(self, X, regimes):
        self.X = X
        self.regimes = regimes
        self.index = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X.shape[1],))
        self.action_space = spaces.Discrete(3)

    def reset(self):
        self.index = 0
        return self.X[self.index]

    def step(self, action):
        classification_reward = 1 if action == self.regimes[self.index] else -1
        self.index += 1
        done = self.index >= len(self.X) - 1
        obs = self.X[self.index] if not done else np.zeros(self.X.shape[1])
        return obs, classification_reward, done, {}

high_env = HighLevelEnv(X_scaled, regime_labels)
check_env(high_env)
high_model = PPO("MlpPolicy", high_env, verbose=1)
high_model.learn(total_timesteps=20000)
high_model.save("high_level_agent")

# === Step 4: Low-Level PPO Agents with Benchmark ===
class LowLevelEnv(gym.Env):
    def __init__(self, returns, window_size=21, lambda_risk=0.5, alpha=0.05):
        self.returns = returns
        self.window_size = window_size
        self.lambda_risk = lambda_risk
        self.alpha = alpha
        self.n_assets = returns.shape[1]
        self.index = window_size
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(window_size, self.n_assets))
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,))

    def reset(self):
        self.index = self.window_size
        return self.returns[self.index - self.window_size:self.index]

    def step(self, action):
        action = np.clip(action, 0, 1)
        action /= np.sum(action + 1e-8)
        agent_return = np.dot(action, self.returns[self.index])
        naive_return = np.mean(self.returns[self.index])

        excess_return = agent_return - naive_return
        historical_returns = self.returns[self.index-self.window_size:self.index] @ action
        cvar_threshold = np.percentile(historical_returns, 100*self.alpha)
        cvar_penalty = max(cvar_threshold - agent_return, 0)

        reward = excess_return - self.lambda_risk * cvar_penalty

        self.index += 1
        done = self.index >= len(self.returns)-1
        obs = self.returns[self.index-self.window_size:self.index] if not done else np.zeros((self.window_size, self.n_assets))

        return obs, reward, done, {}

low_agents = {}
returns_scaled = StandardScaler().fit_transform(data.iloc[:, -29:-1])

for regime in range(3):
    regime_data = returns_scaled[data["Regime"] == regime]
    if len(regime_data) < 50:
        continue

    env = LowLevelEnv(regime_data)
    check_env(env)
    eval_callback = EvalCallback(env, best_model_save_path=f"./best_low_agent_{regime}",
                                 eval_freq=5000, deterministic=True)
    model = PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=50000, callback=eval_callback)
    model.save(f"low_agent_regime_{regime}")
    low_agents[regime] = model

# === Step 5: End-to-End Evaluation ===
evaluation = []
for i in range(len(X_scaled)):
    regime_pred, _ = high_model.predict(X_scaled[i], deterministic=True)
    if regime_pred not in low_agents:
        continue
    low_agent = low_agents[regime_pred]
    stock_obs = returns_scaled[i:i+21]
    if len(stock_obs) < 21:
        continue
    weights, _ = low_agent.predict(stock_obs, deterministic=True)
    weights /= np.sum(weights + 1e-8)
    agent_ret = np.dot(weights, returns_scaled[i])
    naive_ret = np.mean(returns_scaled[i])
    evaluation.append({
        "Date": data["End_Date"].iloc[i],
        "Pred_Regime": regime_pred,
        "True_Regime": regime_labels[i],
        "Agent_Return": agent_ret,
        "Naive_Return": naive_ret,
        "Excess_Return": agent_ret - naive_ret
    })

eval_df = pd.DataFrame(evaluation)
eval_df.to_csv("final_hierarchical_rl_evaluation.csv", index=False)

print("✅ Explicit hierarchical RL pipeline complete. Results saved clearly.")
